In [40]:
import pandas as pd
import numpy as np
import os 
from datetime import datetime
import scipy as sp
import time
from datetime import datetime, date, timedelta

from dateutil.relativedelta import relativedelta

import io
from io import StringIO 
#import datedelta
import calendar
import  csv
import json
import random


from scipy import stats
def add_delta_cols(df):
    #df['delta_vector'] = 0
    #df['delta_max_vector'] = 0

    df['delta_vector'] = 0
    df['delta_max'] = 0
    df['upper_wick'] = 0
    df['lower_wick'] = 0
    
    dv_col = df.columns.get_loc('delta_vector')
    dm_col = df.columns.get_loc('delta_max')
    uw_col = df.columns.get_loc('upper_wick')
    lw_col = df.columns.get_loc('lower_wick')
    
    o_col = df.columns.get_loc('o')
    h_col = df.columns.get_loc('h')
    l_col = df.columns.get_loc('l')
    c_col = df.columns.get_loc('c')
    arr = df.values
    for i in range(arr.shape[0]):
        arr[i,dv_col] = arr[i,c_col] - arr[i,o_col]
        arr[i,dm_col] = arr[i,h_col] - arr[i,l_col]
        if arr[i,c_col] > arr[i,o_col]:
            arr[i,uw_col] = arr[i,h_col] - arr[i,c_col]
            arr[i,lw_col] = arr[i,o_col] - arr[i,l_col]
            
        elif arr[i,c_col] < arr[i,o_col]:
            arr[i,uw_col] = arr[i,h_col] - arr[i,o_col]
            arr[i,lw_col] = arr[i,c_col] - arr[i,l_col]   
            
    df = pd.DataFrame(arr,columns = df.columns)
    df['delta'] = abs(df['delta_vector'])
    
    return df
def load_df(pair = 'EUR_USD'
            ,granularity = 'M5'
            ,start = datetime(2016,1,1,0,0,0)
            ,end = datetime(2022,7,31,0,0,0)):
    def convert_timestamp(df):
        time_col = df.columns.get_loc('time')
        df['minute_col'] = 0
        new_col = df.columns.get_loc('minute_col')
        arr = df.values 
        for i in range(arr.shape[0]):
            arr[i,new_col] = pd.Timestamp(arr[i,time_col]).minute
        return pd.DataFrame(arr,columns = df.columns)
    dir_name = os.getcwd() + '/' + pair + '_' + granularity
    path = dir_name + '/' + str(date(start.year,start.month,start.day)) + '_' + str(date(end.year,end.month,end.day)) + '.csv'
    print('PATH:',path)
    df = pd.read_csv(path)
    df = add_delta_cols(df)
    df = convert_timestamp(df)
    print(df.shape)
    return df
def get_support(df,
                lookup_range = 1500,
                pip_delta = .001
               ):
    print('GETTING SUPPORT INDICATOR')
    print('LOOKUP RANGE AND PIP DELTA:',lookup_range,pip_delta)
    s = time.time()
    c_col = df.columns.get_loc('c')
    o_col = df.columns.get_loc('o')
    l_col = df.columns.get_loc('l')
    h_col = df.columns.get_loc('h')
    d_col = df.columns.get_loc('delta')
    minute_col = df.columns.get_loc('minute_col')
    df['support_indicator'] = 0
    new_col = df.columns.get_loc('support_indicator')
    arr = df.values
    min_count = 0
    c = 0 
    if pip_delta < .01:
        low_spread = 0 #.0001
    else:
        low_spread = 0 #.01
    for i in range(lookup_range,arr.shape[0]):
       # try:
        #DEFINE SUPPORT/RESISTANCE as the MIN or MAX of a lookup range
        df.iloc[i - 11 : i + 1,:]
        if arr[i,c_col] <= min(arr[i - lookup_range : i,c_col]) and arr[i,c_col] <= min(arr[i - 11 : i + 1,l_col]) + low_spread  and (arr[i - 11,o_col] - arr[i,c_col]) >= pip_delta and arr[i,minute_col] == 55:
            arr[i,new_col] = 1

    print('COUNT:',c)

    e = time.time()
    print('TOTAL FUNCTION TIME:',(e-s)/60,' MINUTES')
    df = pd.DataFrame(arr,columns = df.columns)
    print('SHAPE',df[df['support_indicator'] == 1].shape)
    return df

def get_resistance(df,
                lookup_range = 1500,
                    pip_delta = .001
               ):
    print('GETTING RESISTANCE INDICATOR')
    print('LOOKUP RANGE AND PIP DELTA:',lookup_range,pip_delta)
    s = time.time()
    c_col = df.columns.get_loc('c')
    o_col = df.columns.get_loc('o')
    l_col = df.columns.get_loc('l')
    h_col = df.columns.get_loc('h')
    d_col = df.columns.get_loc('delta')
    minute_col = df.columns.get_loc('minute_col')
    df['resistance_indicator'] = 0
    new_col = df.columns.get_loc('resistance_indicator')
    arr = df.values
    min_count = 0
    c = 0 
    if pip_delta < .01:
        low_spread = 0 #.0001
    else:
        low_spread = 0 #.01
    for i in range(lookup_range,arr.shape[0]):
       # try:
        #DEFINE SUPPORT/RESISTANCE as the MIN or MAX of a lookup range
        if arr[i,c_col] >= max(arr[i - lookup_range : i,c_col]) and arr[i,c_col] >= max(arr[i - 11 : i + 1,h_col]) - low_spread and (arr[i,c_col] - arr[i - 11,o_col] ) >= pip_delta and arr[i,minute_col] == 55:
            arr[i,new_col] = 1

    print('COUNT:',c)

    e = time.time()
    print('TOTAL FUNCTION TIME:',(e-s)/60,' MINUTES')
    df = pd.DataFrame(arr,columns = df.columns)
    print('SHAPE',df[df['resistance_indicator'] == 1].shape)
    return df
def strategy_tester_buy(df,col = 'support_indicator',sl = .1,tp = .5,num_trades = 300,risk_size = .01,entry = 'c',pr = True,spread = .01):
    def simulate_results(
        num_trades = 200,
        acc_val = 10000,
        rr = 6,
        risk_size = .02,
        win_rate = .2,
        pr = True):

        start = acc_val
        for i in range(num_trades):
            rand = random.random()
            if rand > win_rate:
                acc_val = acc_val - (acc_val * risk_size)
            else:
                acc_val = acc_val + (acc_val * (risk_size * rr))
        if pr:
            print('STARTING VAL:',round(start),' ENDING VAL:',round(acc_val))
        return round(acc_val)

    col_num = df.columns.get_loc(col)
    h_col = df.columns.get_loc('h')
    l_col = df.columns.get_loc('l')
    c_col = df.columns.get_loc('c')
    entry_col = df.columns.get_loc(entry)
    arr = df.values
    temp_df = df[df[col] == 1]

    trade_list = []
    trade_res = []
    wins = []
    losses = []
    for i in range(arr.shape[0]):

        if arr[i,col_num] == 1:
            val = arr[i,entry_col]
            trade_id = 0
            try:
                for j in range(i + 1,i + 10000):
                    if i not in trade_list:
                        if trade_id == 0:
                            if arr[j,l_col] <= val - sl + spread:
                                trade_res.append(-10)
                                trade_list.append(i)
                                losses.append(i)
                                trade_id = 1

                                break
                            if arr[j,h_col] >= val + tp:
                                trade_res.append(50)
                                trade_list.append(i)
                                wins.append(i)
                                trade_id = 1
                                break
            except:
                pass
    win_rate = len(wins) / len(trade_list)
    rr = (tp - spread) / (sl + spread)
   # num_trades = len(trade_list)
  #  print('WIN RATE',win_rate,' RR:',rr, 'NUM TRADES:',num_trades,' RISK SIZE:',risk_size)
    vals = []


    acc_val = 10000



    for i in range(30):

        final_val = simulate_results(num_trades = num_trades,acc_val = acc_val,rr = rr,risk_size = risk_size,win_rate = win_rate,pr = pr)
        vals.append(final_val)

    return np.median(vals),win_rate,wins,losses

def strategy_tester_sell(df,col = 'support_indicator',sl = .1,tp = .5,num_trades = 300,risk_size = .01,entry = 'c',pr = True,spread = .01):
    def simulate_results(
        num_trades = 200,
        acc_val = 10000,
        rr = 6,
        risk_size = .02,
        win_rate = .2,
        pr = True):

        start = acc_val
        for i in range(num_trades):
            rand = random.random()
            if rand > win_rate:
                acc_val = acc_val - (acc_val * risk_size)
            else:
                acc_val = acc_val + (acc_val * (risk_size * rr))
        if pr:
            print('STARTING VAL:',round(start),' ENDING VAL:',round(acc_val))
        return round(acc_val)

    col_num = df.columns.get_loc(col)
    h_col = df.columns.get_loc('h')
    l_col = df.columns.get_loc('l')
    c_col = df.columns.get_loc('c')
    entry_col = df.columns.get_loc(entry)
    arr = df.values
    temp_df = df[df[col] == 1]

    trade_list = []
    trade_res = []
    wins = []
    losses = []
    for i in range(arr.shape[0]):

        if arr[i,col_num] == 1:
            val = arr[i,entry_col]
            try:
                for j in range(i + 1,i + 10000):
                    if i not in trade_list:
                        if arr[j,h_col] >= val + sl - spread:
                            trade_res.append(-10)
                            trade_list.append(i)
                            losses.append(i)
                            break
                        if arr[j,l_col] <= val - tp:
                            trade_res.append(50)
                            trade_list.append(i)
                            wins.append(i)
                            break
            except:
                pass
    win_rate = len(wins) / len(trade_list)
    rr = (tp - spread) / (sl + spread)
   # num_trades = len(trade_list)
  #  print('WIN RATE',win_rate,' RR:',rr, 'NUM TRADES:',num_trades,' RISK SIZE:',risk_size)
    vals = []


    acc_val = 10000



    for i in range(30):

        final_val = simulate_results(num_trades = num_trades,acc_val = acc_val,rr = rr,risk_size = risk_size,win_rate = win_rate,pr = pr)
        vals.append(final_val)

    return np.median(vals),win_rate,wins,losses
def run_single_pair_sim(pair,
                lookup_range,
                    pip_delta,
                sls,
                tps,
                spread):
    
    def strategy_tester_buy(df,col = 'support_indicator',sl = .1,tp = .5,num_trades = 300,risk_size = .01,entry = 'c',pr = True,spread = .01):
        def simulate_results(
            num_trades = 200,
            acc_val = 10000,
            rr = 6,
            risk_size = .02,
            win_rate = .2,
            pr = True):

            start = acc_val
            for i in range(num_trades):
                rand = random.random()
                if rand > win_rate:
                    acc_val = acc_val - (acc_val * risk_size)
                else:
                    acc_val = acc_val + (acc_val * (risk_size * rr))
            if pr:
                print('STARTING VAL:',round(start),' ENDING VAL:',round(acc_val))
            return round(acc_val)

        col_num = df.columns.get_loc(col)
        h_col = df.columns.get_loc('h')
        l_col = df.columns.get_loc('l')
        c_col = df.columns.get_loc('c')
        entry_col = df.columns.get_loc(entry)
        arr = df.values
        temp_df = df[df[col] == 1]

        trade_list = []
        trade_res = []
        wins = []
        losses = []
        for i in range(arr.shape[0]):

            if arr[i,col_num] == 1:
                val = arr[i,entry_col]
                trade_id = 0
                try:
                    for j in range(i + 1,i + 10000):
                        if i not in trade_list:
                            if trade_id == 0:
                                if arr[j,l_col] <= val - sl + spread:
                                    trade_res.append(-10)
                                    trade_list.append(i)
                                    losses.append(i)
                                    trade_id = 1

                                    break
                                if arr[j,h_col] >= val + tp:
                                    trade_res.append(50)
                                    trade_list.append(i)
                                    wins.append(i)
                                    trade_id = 1
                                    break
                except:
                    pass
        win_rate = len(wins) / len(trade_list)
        rr = (tp - spread) / (sl + spread)
       # num_trades = len(trade_list)
      #  print('WIN RATE',win_rate,' RR:',rr, 'NUM TRADES:',num_trades,' RISK SIZE:',risk_size)
        vals = []


        acc_val = 10000



        for i in range(30):

            final_val = simulate_results(num_trades = num_trades,acc_val = acc_val,rr = rr,risk_size = risk_size,win_rate = win_rate,pr = pr)
            vals.append(final_val)

        return np.median(vals),win_rate,wins,losses

    def strategy_tester_sell(df,col = 'support_indicator',sl = .1,tp = .5,num_trades = 300,risk_size = .01,entry = 'c',pr = True,spread = .01):
        def simulate_results(
            num_trades = 200,
            acc_val = 10000,
            rr = 6,
            risk_size = .02,
            win_rate = .2,
            pr = True):

            start = acc_val
            for i in range(num_trades):
                rand = random.random()
                if rand > win_rate:
                    acc_val = acc_val - (acc_val * risk_size)
                else:
                    acc_val = acc_val + (acc_val * (risk_size * rr))
            if pr:
                print('STARTING VAL:',round(start),' ENDING VAL:',round(acc_val))
            return round(acc_val)

        col_num = df.columns.get_loc(col)
        h_col = df.columns.get_loc('h')
        l_col = df.columns.get_loc('l')
        c_col = df.columns.get_loc('c')
        entry_col = df.columns.get_loc(entry)
        arr = df.values
        temp_df = df[df[col] == 1]

        trade_list = []
        trade_res = []
        wins = []
        losses = []
        for i in range(arr.shape[0]):

            if arr[i,col_num] == 1:
                val = arr[i,entry_col]
                try:
                    for j in range(i + 1,i + 10000):
                        if i not in trade_list:
                            if arr[j,h_col] >= val + sl - spread:
                                trade_res.append(-10)
                                trade_list.append(i)
                                losses.append(i)
                                break
                            if arr[j,l_col] <= val - tp:
                                trade_res.append(50)
                                trade_list.append(i)
                                wins.append(i)
                                break
                except:
                    pass
        win_rate = len(wins) / len(trade_list)
        rr = (tp - spread) / (sl + spread)
       # num_trades = len(trade_list)
      #  print('WIN RATE',win_rate,' RR:',rr, 'NUM TRADES:',num_trades,' RISK SIZE:',risk_size)
        vals = []


        acc_val = 10000



        for i in range(30):

            final_val = simulate_results(num_trades = num_trades,acc_val = acc_val,rr = rr,risk_size = risk_size,win_rate = win_rate,pr = pr)
            vals.append(final_val)

        return np.median(vals),win_rate,wins,losses


    s = time.time()
    df = load_df(pair = pair,granularity ='M5',start = datetime(2016,1,1,0,0,0),end = datetime(2022,7,31,0,0,0))
    print(df.shape)
    df_ = df.copy()

    print('RUNNING SINGLE PARAMETER LOOP FOR PARAMS:',
          'INSTRUMENT:',pair,
          'lookup_range',lookup_range)

    df = get_support(df,
                    lookup_range = lookup_range,
                     pip_delta = pip_delta
                    )

    df_ = get_resistance(df_,
                    lookup_range = lookup_range,
                     pip_delta = pip_delta
                   )


    cols = ['instrument',
           'buy_support_val',
           'sell_resistance_val',
           'buy_resistance_val',
           'sell_support_val',
            'max_ending_val',
            'sl',
            'tp',
            'RR',
           'win_rate1',
           'win_rate2',
           'win_rate3',
           'win_rate4',
            'lookup_range',
            'support_number_of_trades',
            'resistance_number_of_trades'
          ]

    trade_strategy = 'support_indicator'
    trade_strategy2 = 'resistance_indicator'

    #trade_strategy = 'resistance_indicatorc'
    num_trades = 400
    risk_size = .01

    starting_val = 10000
    instrument = pair
    timeframe = 'M5'
    temp_lst = []
    actual_number_of_trades = df[df[trade_strategy] == 1].shape[0]
    actual_number_of_trades2 = df_[df_[trade_strategy2] == 1].shape[0]


    for sl in sls:
        for tp in tps:
            if actual_number_of_trades == 0:
                ending_val,win_rate,wins,losses = 0,0,0,0
                ending_val4,win_rate4,wins4,losses4 = 0,0,0,0
            else:
                ending_val,win_rate,wins,losses = strategy_tester_buy(df,col = trade_strategy,sl = sl,tp = tp,num_trades = num_trades,risk_size = risk_size,entry = 'c',pr = False,spread = spread)
                ending_val4,win_rate4,wins4,losses4 = strategy_tester_sell(df,col = trade_strategy,sl = sl,tp = tp,num_trades = num_trades,risk_size = risk_size,entry = 'c',pr = False,spread = spread)   
                
            if actual_number_of_trades2 == 0:
                ending_val2,win_rate2,wins2,losses2 = 0,0,0,0
                ending_val3,win_rate3,wins3,losses3 = 0,0,0,0
            else:
                ending_val2,win_rate2,wins2,losses2 = strategy_tester_sell(df_,col = trade_strategy2,sl = sl,tp = tp,num_trades = num_trades,risk_size = risk_size,entry = 'c',pr = False,spread = spread)
                ending_val3,win_rate3,wins3,losses3 = strategy_tester_buy(df_,col = trade_strategy2,sl = sl,tp = tp,num_trades = num_trades,risk_size = risk_size,entry = 'c',pr = False,spread = spread) 
                
                
            max_ending_val = max(ending_val,
                   ending_val2,
                   ending_val3,
                   ending_val4)
            lst = [instrument,
                   ending_val,
                   ending_val2,
                   ending_val3,
                   ending_val4,
                   max_ending_val,
                    sl,
                    tp,
                    round(tp / sl,4),
                   round(win_rate,4),
                   round(win_rate2,4),
                   round(win_rate3,4),
                   round(win_rate4,4),
                    lookup_range,
                   actual_number_of_trades,
                   actual_number_of_trades2
                  ]
            temp_lst.append(lst)

    final_df = pd.DataFrame(temp_lst,columns = cols)
    final_df = final_df.sort_values(by = 'max_ending_val',ascending = False)
    
    e = time.time()
    best_buy_support = final_df['buy_support_val'].max()
    best_sell_resistance = final_df['sell_resistance_val'].max()
    best_buy_resistance = final_df['buy_resistance_val'].max()
    best_sell_support = final_df['sell_support_val'].max()
    max_val_all = final_df['max_ending_val'].max()
    print('MAX VAL:',round(max_val_all),'||best buy support:',round(best_buy_support),'||best sell resistance:',
          round(best_sell_resistance),'||best buy resistance:',round(best_buy_resistance),'||best sell support:',round(best_sell_support))


    print('TOTAL SINGLE PAIR SIM TIME:',round((e-s)/60,1),'MINUTES')
    return final_df

def run_all_pairs_sim_loop(
                    lookup_range,
    pip_delta,
                    spread,
                    strategy_name = 'large_candle_sim_results'
                    ):
    dir_name = os.getcwd() + '/' + strategy_name 
    print(dir_name)
    isdir = os.path.isdir(dir_name) 
    if isdir:
        pass
    else:
        print('MAKING DIRECTORY:',dir_name)
        os.mkdir(dir_name) 
        
        
    dir_name = os.getcwd() + '/' + strategy_name + '/' + str(lookup_range)
    print(dir_name)
    isdir = os.path.isdir(dir_name) 
    if isdir:
        pass
    else:
        print('MAKING DIRECTORY:',dir_name)
        os.mkdir(dir_name)         
        
        

    if len(os.listdir(dir_name)) > 0:
        print('file already exists in this directory, passing...')
    else:
        print('file does not exist, running sim')


        all_pairs=['EUR_USD','GBP_JPY','USD_JPY','EUR_JPY','AUD_JPY','CAD_JPY','CHF_JPY',
                   'GBP_USD','USD_CAD','USD_CHF','AUD_USD',
                   'NZD_USD','AUD_CAD','AUD_CHF','AUD_NZD','CAD_CHF',
                   'EUR_AUD','EUR_CAD','EUR_CHF','NZD_CAD','NZD_CHF'] 


        final_df = pd.DataFrame([0])    
        for pair in all_pairs:
            print(pair)  

            if 'JPY' in pair:
                sls = [.05,.075,.1,.125,.15,.2,.25]
                tps = [.1,.15,.2,.25,.3,.4,.5,.7,.9,1.1,1.3,1.5]
                spread = .01
                pip_delta_ = pip_delta * 100
            else:
                sls = [.0005,.00075,.001,.00125,.0015,.002,.0025]
                tps = [.001,.0015,.002,.0025,.003,.004,.005,.007,.009,.011,.013,.015]
                spread = .0001      
                pip_delta_ = pip_delta

            df = run_single_pair_sim(pair,
                            lookup_range,
                            pip_delta_,
                            sls,
                            tps,
                            spread)    
            if final_df.shape[0] == 1:
                final_df = df
            else:
                final_df = pd.concat([final_df,df],axis = 0)

        final_df = final_df.sort_values(by = 'max_ending_val',ascending = False)    

        gb = final_df.groupby('instrument').agg({'max_ending_val':max,'buy_support_val':max,'sell_resistance_val':max,'buy_resistance_val':max,'sell_support_val':max})
        gb = gb.reset_index()
        gb = gb.sort_values(by = 'max_ending_val',ascending = False)    
        for i in range(gb.shape[0]):
            print('pair',gb['instrument'].iloc[i],'||MAX VAL',gb['max_ending_val'].iloc[i],'||best buy support:',gb['buy_support_val'].iloc[i],'||best sell resistance:',
                  gb['sell_resistance_val'].iloc[i],'||best buy resistance:',gb['buy_resistance_val'].iloc[i],'||best sell support:',gb['sell_support_val'].iloc[i])

        print('MEDIANS:',np.median(gb['max_ending_val']),np.median(gb['buy_support_val']),np.median(gb['sell_resistance_val']),np.median(gb['buy_resistance_val']),np.median(gb['sell_support_val']))

        path = dir_name + '/' + str(round(np.median(gb['max_ending_val']))) + '_' + str(round(np.median(gb['buy_support_val']))) + '_' + str(round(np.median(gb['sell_resistance_val']))) + '_' + str(round(np.median(gb['buy_resistance_val']))) + '_' + str(round(np.median(gb['sell_support_val'])))
        path = path + '.csv'
        print('FINAL PATH:',path)
        print('SAVING DATA')
        final_df.to_csv(path,index = False)
        print()
        print()
        print()
        print()
        print()
        
        
lookup_range = 100
spread = .0001
pip_delta = .0015
run_all_pairs_sim_loop(
                    lookup_range,
                    pip_delta = pip_delta,
                    spread = spread,
                    strategy_name = 'no_wick_sim_results'
                    )


/Users/aidanmcconnell/Documents/MARKET_ANALYTICS/forex/Macro/no_wick_sim_results
/Users/aidanmcconnell/Documents/MARKET_ANALYTICS/forex/Macro/no_wick_sim_results/100
file does not exist, running sim
EUR_USD
PATH: /Users/aidanmcconnell/Documents/MARKET_ANALYTICS/forex/Macro/EUR_USD_M5/2016-01-01_2022-07-31.csv
(490023, 26)
(490023, 26)
RUNNING SINGLE PARAMETER LOOP FOR PARAMS: INSTRUMENT: EUR_USD lookup_range 100
GETTING SUPPORT INDICATOR
LOOKUP RANGE AND PIP DELTA: 100 0.0015
COUNT: 0
TOTAL FUNCTION TIME: 0.2897636850674947  MINUTES
SHAPE (46, 27)
GETTING RESISTANCE INDICATOR
LOOKUP RANGE AND PIP DELTA: 100 0.0015
COUNT: 0
TOTAL FUNCTION TIME: 0.027445085843404136  MINUTES
SHAPE (29, 27)
MAX VAL: 256690 ||best buy support: 256690 ||best sell resistance: 11384 ||best buy resistance: 11118 ||best sell support: 5220
TOTAL SINGLE PAIR SIM TIME: 0.8 MINUTES
GBP_JPY
PATH: /Users/aidanmcconnell/Documents/MARKET_ANALYTICS/forex/Macro/GBP_JPY_M5/2016-01-01_2022-07-31.csv
(491385, 26)
(491385, 2

In [47]:
def get_support(df,
                lookup_range = 1500,
                pip_delta = .001
               ):
    print('GETTING SUPPORT INDICATOR')
    print('LOOKUP RANGE AND PIP DELTA:',lookup_range,pip_delta)
    s = time.time()
    c_col = df.columns.get_loc('c')
    o_col = df.columns.get_loc('o')
    l_col = df.columns.get_loc('l')
    h_col = df.columns.get_loc('h')
    d_col = df.columns.get_loc('delta')
    minute_col = df.columns.get_loc('minute_col')
    df['support_indicator'] = 0
    new_col = df.columns.get_loc('support_indicator')
    arr = df.values
    min_count = 0
    c = 0 
    if pip_delta < .01:
        low_spread = 0 #.0001
        next_candle = .00075
    else:
        low_spread = 0 #.01
        next_candle = .075
    for i in range(lookup_range,arr.shape[0]):
       # try:
        #DEFINE SUPPORT/RESISTANCE as the MIN or MAX of a lookup range
        df.iloc[i - 11 : i + 1,:]
        if arr[i,c_col] <= min(arr[i - lookup_range : i,c_col]) and arr[i,c_col] <= min(arr[i - 11 : i + 1,l_col]) + low_spread  and (arr[i - 11,o_col] - arr[i,c_col]) >= pip_delta and (arr[i,minute_col] == 55 or arr[i,minute_col] == 25) and arr[i,c_col] - min(arr[i + 1 : i + 6,l_col]) >= next_candle:
            arr[i,new_col] = 1
            arr[i,c_col] = arr[i,c_col] - next_candle

    print('COUNT:',c)

    e = time.time()
    print('TOTAL FUNCTION TIME:',(e-s)/60,' MINUTES')
    df = pd.DataFrame(arr,columns = df.columns)
    print('SHAPE',df[df['support_indicator'] == 1].shape)
    return df

def get_resistance(df,
                lookup_range = 1500,
                    pip_delta = .001
               ):
    print('GETTING RESISTANCE INDICATOR')
    print('LOOKUP RANGE AND PIP DELTA:',lookup_range,pip_delta)
    s = time.time()
    c_col = df.columns.get_loc('c')
    o_col = df.columns.get_loc('o')
    l_col = df.columns.get_loc('l')
    h_col = df.columns.get_loc('h')
    d_col = df.columns.get_loc('delta')
    minute_col = df.columns.get_loc('minute_col')
    df['resistance_indicator'] = 0
    new_col = df.columns.get_loc('resistance_indicator')
    arr = df.values
    min_count = 0
    c = 0 
    if pip_delta < .01:
        low_spread = 0 #.0001
        next_candle = .00075
    else:
        low_spread = 0 #.01
        next_candle = .075
    for i in range(lookup_range,arr.shape[0]):
       # try:
        #DEFINE SUPPORT/RESISTANCE as the MIN or MAX of a lookup range
        if arr[i,c_col] >= max(arr[i - lookup_range : i,c_col]) and arr[i,c_col] >= max(arr[i - 11 : i + 1,h_col]) - low_spread and (arr[i,c_col] - arr[i - 11,o_col] ) >= pip_delta and (arr[i,minute_col] == 55 or arr[i,minute_col] == 25)  and max(arr[i + 1 : i + 6,h_col]) - arr[i,c_col] >= next_candle:
            arr[i,new_col] = 1
            arr[i,c_col] = arr[i,c_col] + next_candle

    print('COUNT:',c)

    e = time.time()
    print('TOTAL FUNCTION TIME:',(e-s)/60,' MINUTES')
    df = pd.DataFrame(arr,columns = df.columns)
    print('SHAPE',df[df['resistance_indicator'] == 1].shape)
    return df

def run_single_pair_sim(pair,
                lookup_range,
                    pip_delta,
                sls,
                tps,
                spread):
    
    def strategy_tester_buy(df,col = 'support_indicator',sl = .1,tp = .5,num_trades = 300,risk_size = .01,entry = 'c',pr = True,spread = .01):
        def simulate_results(
            num_trades = 200,
            acc_val = 10000,
            rr = 6,
            risk_size = .02,
            win_rate = .2,
            pr = True):

            start = acc_val
            for i in range(num_trades):
                rand = random.random()
                if rand > win_rate:
                    acc_val = acc_val - (acc_val * risk_size)
                else:
                    acc_val = acc_val + (acc_val * (risk_size * rr))
            if pr:
                print('STARTING VAL:',round(start),' ENDING VAL:',round(acc_val))
            return round(acc_val)

        col_num = df.columns.get_loc(col)
        h_col = df.columns.get_loc('h')
        l_col = df.columns.get_loc('l')
        c_col = df.columns.get_loc('c')
        entry_col = df.columns.get_loc(entry)
        arr = df.values
        temp_df = df[df[col] == 1]

        trade_list = []
        trade_res = []
        wins = []
        losses = []
        for i in range(arr.shape[0]):

            if arr[i,col_num] == 1:
                val = arr[i,entry_col]
                trade_id = 0
                try:
                    for j in range(i + 1,i + 10000):
                        if i not in trade_list:
                            if trade_id == 0:
                                if arr[j,l_col] <= val - sl + spread:
                                    trade_res.append(-10)
                                    trade_list.append(i)
                                    losses.append(i)
                                    trade_id = 1

                                    break
                                if arr[j,h_col] >= val + tp:
                                    trade_res.append(50)
                                    trade_list.append(i)
                                    wins.append(i)
                                    trade_id = 1
                                    break
                except:
                    pass
        win_rate = len(wins) / len(trade_list)
        rr = (tp - spread) / (sl + spread)
       # num_trades = len(trade_list)
      #  print('WIN RATE',win_rate,' RR:',rr, 'NUM TRADES:',num_trades,' RISK SIZE:',risk_size)
        vals = []


        acc_val = 10000



        for i in range(30):

            final_val = simulate_results(num_trades = num_trades,acc_val = acc_val,rr = rr,risk_size = risk_size,win_rate = win_rate,pr = pr)
            vals.append(final_val)

        return np.median(vals),win_rate,wins,losses

    def strategy_tester_sell(df,col = 'support_indicator',sl = .1,tp = .5,num_trades = 300,risk_size = .01,entry = 'c',pr = True,spread = .01):
        def simulate_results(
            num_trades = 200,
            acc_val = 10000,
            rr = 6,
            risk_size = .02,
            win_rate = .2,
            pr = True):

            start = acc_val
            for i in range(num_trades):
                rand = random.random()
                if rand > win_rate:
                    acc_val = acc_val - (acc_val * risk_size)
                else:
                    acc_val = acc_val + (acc_val * (risk_size * rr))
            if pr:
                print('STARTING VAL:',round(start),' ENDING VAL:',round(acc_val))
            return round(acc_val)

        col_num = df.columns.get_loc(col)
        h_col = df.columns.get_loc('h')
        l_col = df.columns.get_loc('l')
        c_col = df.columns.get_loc('c')
        entry_col = df.columns.get_loc(entry)
        arr = df.values
        temp_df = df[df[col] == 1]

        trade_list = []
        trade_res = []
        wins = []
        losses = []
        for i in range(arr.shape[0]):

            if arr[i,col_num] == 1:
                val = arr[i,entry_col]
                try:
                    for j in range(i + 1,i + 10000):
                        if i not in trade_list:
                            if arr[j,h_col] >= val + sl - spread:
                                trade_res.append(-10)
                                trade_list.append(i)
                                losses.append(i)
                                break
                            if arr[j,l_col] <= val - tp:
                                trade_res.append(50)
                                trade_list.append(i)
                                wins.append(i)
                                break
                except:
                    pass
        win_rate = len(wins) / len(trade_list)
        rr = (tp - spread) / (sl + spread)
       # num_trades = len(trade_list)
      #  print('WIN RATE',win_rate,' RR:',rr, 'NUM TRADES:',num_trades,' RISK SIZE:',risk_size)
        vals = []


        acc_val = 10000



        for i in range(30):

            final_val = simulate_results(num_trades = num_trades,acc_val = acc_val,rr = rr,risk_size = risk_size,win_rate = win_rate,pr = pr)
            vals.append(final_val)

        return np.median(vals),win_rate,wins,losses


    s = time.time()
    df = load_df(pair = pair,granularity ='M5',start = datetime(2016,1,1,0,0,0),end = datetime(2022,7,31,0,0,0))
    print(df.shape)
    df_ = df.copy()

    print('RUNNING SINGLE PARAMETER LOOP FOR PARAMS:',
          'INSTRUMENT:',pair,
          'lookup_range',lookup_range)

    df = get_support(df,
                    lookup_range = lookup_range,
                     pip_delta = pip_delta
                    )

    df_ = get_resistance(df_,
                    lookup_range = lookup_range,
                     pip_delta = pip_delta
                   )


    cols = ['instrument',
           'buy_support_val',
           'sell_resistance_val',
           'buy_resistance_val',
           'sell_support_val',
            'max_ending_val',
            'sl',
            'tp',
            'RR',
           'win_rate1',
           'win_rate2',
           'win_rate3',
           'win_rate4',
            'lookup_range',
            'support_number_of_trades',
            'resistance_number_of_trades'
          ]

    trade_strategy = 'support_indicator'
    trade_strategy2 = 'resistance_indicator'

    #trade_strategy = 'resistance_indicatorc'
    num_trades = 400
    risk_size = .01

    starting_val = 10000
    instrument = pair
    timeframe = 'M5'
    temp_lst = []
    actual_number_of_trades = df[df[trade_strategy] == 1].shape[0]
    actual_number_of_trades2 = df_[df_[trade_strategy2] == 1].shape[0]


    for sl in sls:
        for tp in tps:
            if actual_number_of_trades == 0:
                ending_val,win_rate,wins,losses = 0,0,0,0
                ending_val4,win_rate4,wins4,losses4 = 0,0,0,0
            else:
                ending_val,win_rate,wins,losses = strategy_tester_buy(df,col = trade_strategy,sl = sl,tp = tp,num_trades = num_trades,risk_size = risk_size,entry = 'c',pr = False,spread = spread)
                ending_val4,win_rate4,wins4,losses4 = strategy_tester_sell(df,col = trade_strategy,sl = sl,tp = tp,num_trades = num_trades,risk_size = risk_size,entry = 'c',pr = False,spread = spread)   
                
            if actual_number_of_trades2 == 0:
                ending_val2,win_rate2,wins2,losses2 = 0,0,0,0
                ending_val3,win_rate3,wins3,losses3 = 0,0,0,0
            else:
                ending_val2,win_rate2,wins2,losses2 = strategy_tester_sell(df_,col = trade_strategy2,sl = sl,tp = tp,num_trades = num_trades,risk_size = risk_size,entry = 'c',pr = False,spread = spread)
                ending_val3,win_rate3,wins3,losses3 = strategy_tester_buy(df_,col = trade_strategy2,sl = sl,tp = tp,num_trades = num_trades,risk_size = risk_size,entry = 'c',pr = False,spread = spread) 
                
                
            max_ending_val = max(ending_val,
                   ending_val2,
                   ending_val3,
                   ending_val4)
            lst = [instrument,
                   ending_val,
                   ending_val2,
                   ending_val3,
                   ending_val4,
                   max_ending_val,
                    sl,
                    tp,
                    round(tp / sl,4),
                   round(win_rate,4),
                   round(win_rate2,4),
                   round(win_rate3,4),
                   round(win_rate4,4),
                    lookup_range,
                   actual_number_of_trades,
                   actual_number_of_trades2
                  ]
            temp_lst.append(lst)

    final_df = pd.DataFrame(temp_lst,columns = cols)
    final_df = final_df.sort_values(by = 'max_ending_val',ascending = False)
    
    e = time.time()
    best_buy_support = final_df['buy_support_val'].max()
    best_sell_resistance = final_df['sell_resistance_val'].max()
    best_buy_resistance = final_df['buy_resistance_val'].max()
    best_sell_support = final_df['sell_support_val'].max()
    max_val_all = final_df['max_ending_val'].max()
    print('MAX VAL:',round(max_val_all),'||best buy support:',round(best_buy_support),'||best sell resistance:',
          round(best_sell_resistance),'||best buy resistance:',round(best_buy_resistance),'||best sell support:',round(best_sell_support))


    print('TOTAL SINGLE PAIR SIM TIME:',round((e-s)/60,1),'MINUTES')
    return final_df

def run_all_pairs_sim_loop(
                    lookup_range,
    pip_delta,
                    spread,
                    strategy_name = 'large_candle_sim_results'
                    ):
    dir_name = os.getcwd() + '/' + strategy_name 
    print(dir_name)
    isdir = os.path.isdir(dir_name) 
    if isdir:
        pass
    else:
        print('MAKING DIRECTORY:',dir_name)
        os.mkdir(dir_name) 
        
        
    dir_name = os.getcwd() + '/' + strategy_name + '/' + str(lookup_range)
    print(dir_name)
    isdir = os.path.isdir(dir_name) 
    if isdir:
        pass
    else:
        print('MAKING DIRECTORY:',dir_name)
        os.mkdir(dir_name)         
        
        

    if len(os.listdir(dir_name)) > 0:
        print('file already exists in this directory, passing...')
    else:
        print('file does not exist, running sim')


        all_pairs=['EUR_USD','GBP_JPY','USD_JPY','EUR_JPY','AUD_JPY','CAD_JPY','CHF_JPY',
                   'GBP_USD','USD_CAD','USD_CHF','AUD_USD',
                   'NZD_USD','AUD_CAD','AUD_CHF','AUD_NZD','CAD_CHF',
                   'EUR_AUD','EUR_CAD','EUR_CHF','NZD_CAD','NZD_CHF'] 


        final_df = pd.DataFrame([0])    
        for pair in all_pairs:
            print(pair)  

            if 'JPY' in pair:
                sls = [.05,.075,.1,.125,.15,.2,.25]
                tps = [.1,.15,.2,.25,.3,.4,.5,.7,.9,1.1,1.3,1.5]
                spread = .01
                pip_delta_ = pip_delta * 100
            else:
                sls = [.0005,.00075,.001,.00125,.0015,.002,.0025]
                tps = [.001,.0015,.002,.0025,.003,.004,.005,.007,.009,.011,.013,.015]
                spread = .0001      
                pip_delta_ = pip_delta

            df = run_single_pair_sim(pair,
                            lookup_range,
                            pip_delta_,
                            sls,
                            tps,
                            spread)    
            if final_df.shape[0] == 1:
                final_df = df
            else:
                final_df = pd.concat([final_df,df],axis = 0)

        final_df = final_df.sort_values(by = 'max_ending_val',ascending = False)    

        gb = final_df.groupby('instrument').agg({'max_ending_val':max,'buy_support_val':max,'sell_resistance_val':max,'buy_resistance_val':max,'sell_support_val':max})
        gb = gb.reset_index()
        gb = gb.sort_values(by = 'max_ending_val',ascending = False)    
        for i in range(gb.shape[0]):
            print('pair',gb['instrument'].iloc[i],'||MAX VAL',gb['max_ending_val'].iloc[i],'||best buy support:',gb['buy_support_val'].iloc[i],'||best sell resistance:',
                  gb['sell_resistance_val'].iloc[i],'||best buy resistance:',gb['buy_resistance_val'].iloc[i],'||best sell support:',gb['sell_support_val'].iloc[i])

        print('MEDIANS:',np.median(gb['max_ending_val']),np.median(gb['buy_support_val']),np.median(gb['sell_resistance_val']),np.median(gb['buy_resistance_val']),np.median(gb['sell_support_val']))

        path = dir_name + '/' + str(round(np.median(gb['max_ending_val']))) + '_' + str(round(np.median(gb['buy_support_val']))) + '_' + str(round(np.median(gb['sell_resistance_val']))) + '_' + str(round(np.median(gb['buy_resistance_val']))) + '_' + str(round(np.median(gb['sell_support_val'])))
        path = path + '.csv'
        print('FINAL PATH:',path)
        print('SAVING DATA')
        final_df.to_csv(path,index = False)
        print()
        print()
        print()
        print()
        print()
        
        
lookup_range = 75
spread = .0001
pip_delta = .0015
run_all_pairs_sim_loop(
                    lookup_range,
                    pip_delta = pip_delta,
                    spread = spread,
                    strategy_name = 'no_wick_5_on_next_' + str(pip_delta) + '_sim_results'
                    )




/Users/aidanmcconnell/Documents/MARKET_ANALYTICS/forex/Macro/no_wick_5_on_next_0.0015_sim_results
/Users/aidanmcconnell/Documents/MARKET_ANALYTICS/forex/Macro/no_wick_5_on_next_0.0015_sim_results/75
file does not exist, running sim
EUR_USD
PATH: /Users/aidanmcconnell/Documents/MARKET_ANALYTICS/forex/Macro/EUR_USD_M5/2016-01-01_2022-07-31.csv
(490023, 26)
(490023, 26)
RUNNING SINGLE PARAMETER LOOP FOR PARAMS: INSTRUMENT: EUR_USD lookup_range 75
GETTING SUPPORT INDICATOR
LOOKUP RANGE AND PIP DELTA: 75 0.0015
COUNT: 0
TOTAL FUNCTION TIME: 0.32652196486790974  MINUTES
SHAPE (20, 27)
GETTING RESISTANCE INDICATOR
LOOKUP RANGE AND PIP DELTA: 75 0.0015
COUNT: 0
TOTAL FUNCTION TIME: 0.02665170431137085  MINUTES
SHAPE (22, 27)
MAX VAL: 177159482 ||best buy support: 177159482 ||best sell resistance: 66220 ||best buy resistance: 15270 ||best sell support: 37300
TOTAL SINGLE PAIR SIM TIME: 0.9 MINUTES
GBP_JPY
PATH: /Users/aidanmcconnell/Documents/MARKET_ANALYTICS/forex/Macro/GBP_JPY_M5/2016-01-01_2

In [50]:
def get_support(df,
                lookup_range = 1500,
                pip_delta = .001
               ):
    print('GETTING SUPPORT INDICATOR')
    print('LOOKUP RANGE AND PIP DELTA:',lookup_range,pip_delta)
    s = time.time()
    c_col = df.columns.get_loc('c')
    o_col = df.columns.get_loc('o')
    l_col = df.columns.get_loc('l')
    h_col = df.columns.get_loc('h')
    d_col = df.columns.get_loc('delta')
    minute_col = df.columns.get_loc('minute_col')
    df['support_indicator'] = 0
    new_col = df.columns.get_loc('support_indicator')
    arr = df.values
    min_count = 0
    c = 0 
    if pip_delta < .01:
        low_spread = 0 #.0001
        next_candle = .0007
    else:
        low_spread = 0 #.01
        next_candle = .07
    for i in range(lookup_range,arr.shape[0]):
       # try:
        #DEFINE SUPPORT/RESISTANCE as the MIN or MAX of a lookup range
        df.iloc[i - 11 : i + 1,:]
        if arr[i,c_col] <= min(arr[i - lookup_range : i,c_col]) and arr[i,c_col] <= min(arr[i - 11 : i + 1,l_col]) + low_spread  and (arr[i - 11,o_col] - arr[i,c_col]) >= pip_delta and (arr[i,minute_col] == 55 or arr[i,minute_col] == 25) and arr[i,c_col] - min(arr[i + 1 : i + 6,l_col]) >= next_candle:
            arr[i,new_col] = 1
            arr[i,c_col] = arr[i,c_col] - next_candle

    print('COUNT:',c)

    e = time.time()
    print('TOTAL FUNCTION TIME:',(e-s)/60,' MINUTES')
    df = pd.DataFrame(arr,columns = df.columns)
    print('SHAPE',df[df['support_indicator'] == 1].shape)
    return df

def get_resistance(df,
                lookup_range = 1500,
                    pip_delta = .001
               ):
    print('GETTING RESISTANCE INDICATOR')
    print('LOOKUP RANGE AND PIP DELTA:',lookup_range,pip_delta)
    s = time.time()
    c_col = df.columns.get_loc('c')
    o_col = df.columns.get_loc('o')
    l_col = df.columns.get_loc('l')
    h_col = df.columns.get_loc('h')
    d_col = df.columns.get_loc('delta')
    minute_col = df.columns.get_loc('minute_col')
    df['resistance_indicator'] = 0
    new_col = df.columns.get_loc('resistance_indicator')
    arr = df.values
    min_count = 0
    c = 0 
    if pip_delta < .01:
        low_spread = 0 #.0001
        next_candle = .0007
    else:
        low_spread = 0 #.01
        next_candle = .07
    for i in range(lookup_range,arr.shape[0]):
       # try:
        #DEFINE SUPPORT/RESISTANCE as the MIN or MAX of a lookup range
        if arr[i,c_col] >= max(arr[i - lookup_range : i,c_col]) and arr[i,c_col] >= max(arr[i - 11 : i + 1,h_col]) - low_spread and (arr[i,c_col] - arr[i - 11,o_col] ) >= pip_delta and (arr[i,minute_col] == 55 or arr[i,minute_col] == 25)  and max(arr[i + 1 : i + 6,h_col]) - arr[i,c_col] >= next_candle:
            arr[i,new_col] = 1
            arr[i,c_col] = arr[i,c_col] + next_candle

    print('COUNT:',c)

    e = time.time()
    print('TOTAL FUNCTION TIME:',(e-s)/60,' MINUTES')
    df = pd.DataFrame(arr,columns = df.columns)
    print('SHAPE',df[df['resistance_indicator'] == 1].shape)
    return df

def run_single_pair_sim(pair,
                lookup_range,
                    pip_delta,
                sls,
                tps,
                spread):
    
    def strategy_tester_buy(df,col = 'support_indicator',sl = .1,tp = .5,num_trades = 300,risk_size = .01,entry = 'c',pr = True,spread = .01):
        def simulate_results(
            num_trades = 200,
            acc_val = 10000,
            rr = 6,
            risk_size = .02,
            win_rate = .2,
            pr = True):

            start = acc_val
            for i in range(num_trades):
                rand = random.random()
                if rand > win_rate:
                    acc_val = acc_val - (acc_val * risk_size)
                else:
                    acc_val = acc_val + (acc_val * (risk_size * rr))
            if pr:
                print('STARTING VAL:',round(start),' ENDING VAL:',round(acc_val))
            return round(acc_val)

        col_num = df.columns.get_loc(col)
        h_col = df.columns.get_loc('h')
        l_col = df.columns.get_loc('l')
        c_col = df.columns.get_loc('c')
        entry_col = df.columns.get_loc(entry)
        arr = df.values
        temp_df = df[df[col] == 1]

        trade_list = []
        trade_res = []
        wins = []
        losses = []
        for i in range(arr.shape[0]):

            if arr[i,col_num] == 1:
                val = arr[i,entry_col]
                trade_id = 0
                try:
                    for j in range(i + 1,i + 10000):
                        if i not in trade_list:
                            if trade_id == 0:
                                if arr[j,l_col] <= val - sl + spread:
                                    trade_res.append(-10)
                                    trade_list.append(i)
                                    losses.append(i)
                                    trade_id = 1

                                    break
                                if arr[j,h_col] >= val + tp:
                                    trade_res.append(50)
                                    trade_list.append(i)
                                    wins.append(i)
                                    trade_id = 1
                                    break
                except:
                    pass
        win_rate = len(wins) / len(trade_list)
        rr = (tp - spread) / (sl + spread)
       # num_trades = len(trade_list)
      #  print('WIN RATE',win_rate,' RR:',rr, 'NUM TRADES:',num_trades,' RISK SIZE:',risk_size)
        vals = []


        acc_val = 10000



        for i in range(30):

            final_val = simulate_results(num_trades = num_trades,acc_val = acc_val,rr = rr,risk_size = risk_size,win_rate = win_rate,pr = pr)
            vals.append(final_val)

        return np.median(vals),win_rate,wins,losses

    def strategy_tester_sell(df,col = 'support_indicator',sl = .1,tp = .5,num_trades = 300,risk_size = .01,entry = 'c',pr = True,spread = .01):
        def simulate_results(
            num_trades = 200,
            acc_val = 10000,
            rr = 6,
            risk_size = .02,
            win_rate = .2,
            pr = True):

            start = acc_val
            for i in range(num_trades):
                rand = random.random()
                if rand > win_rate:
                    acc_val = acc_val - (acc_val * risk_size)
                else:
                    acc_val = acc_val + (acc_val * (risk_size * rr))
            if pr:
                print('STARTING VAL:',round(start),' ENDING VAL:',round(acc_val))
            return round(acc_val)

        col_num = df.columns.get_loc(col)
        h_col = df.columns.get_loc('h')
        l_col = df.columns.get_loc('l')
        c_col = df.columns.get_loc('c')
        entry_col = df.columns.get_loc(entry)
        arr = df.values
        temp_df = df[df[col] == 1]

        trade_list = []
        trade_res = []
        wins = []
        losses = []
        for i in range(arr.shape[0]):

            if arr[i,col_num] == 1:
                val = arr[i,entry_col]
                try:
                    for j in range(i + 1,i + 10000):
                        if i not in trade_list:
                            if arr[j,h_col] >= val + sl - spread:
                                trade_res.append(-10)
                                trade_list.append(i)
                                losses.append(i)
                                break
                            if arr[j,l_col] <= val - tp:
                                trade_res.append(50)
                                trade_list.append(i)
                                wins.append(i)
                                break
                except:
                    pass
        win_rate = len(wins) / len(trade_list)
        rr = (tp - spread) / (sl + spread)
       # num_trades = len(trade_list)
      #  print('WIN RATE',win_rate,' RR:',rr, 'NUM TRADES:',num_trades,' RISK SIZE:',risk_size)
        vals = []


        acc_val = 10000



        for i in range(30):

            final_val = simulate_results(num_trades = num_trades,acc_val = acc_val,rr = rr,risk_size = risk_size,win_rate = win_rate,pr = pr)
            vals.append(final_val)

        return np.median(vals),win_rate,wins,losses


    s = time.time()
    df = load_df(pair = pair,granularity ='M5',start = datetime(2016,1,1,0,0,0),end = datetime(2022,7,31,0,0,0))
    print(df.shape)
    df_ = df.copy()

    print('RUNNING SINGLE PARAMETER LOOP FOR PARAMS:',
          'INSTRUMENT:',pair,
          'lookup_range',lookup_range)

    df = get_support(df,
                    lookup_range = lookup_range,
                     pip_delta = pip_delta
                    )

    df_ = get_resistance(df_,
                    lookup_range = lookup_range,
                     pip_delta = pip_delta
                   )


    cols = ['instrument',
           'buy_support_val',
           'sell_resistance_val',
           'buy_resistance_val',
           'sell_support_val',
            'max_ending_val',
            'sl',
            'tp',
            'RR',
           'win_rate1',
           'win_rate2',
           'win_rate3',
           'win_rate4',
            'lookup_range',
            'support_number_of_trades',
            'resistance_number_of_trades'
          ]

    trade_strategy = 'support_indicator'
    trade_strategy2 = 'resistance_indicator'

    #trade_strategy = 'resistance_indicatorc'
    num_trades = 400
    risk_size = .01

    starting_val = 10000
    instrument = pair
    timeframe = 'M5'
    temp_lst = []
    actual_number_of_trades = df[df[trade_strategy] == 1].shape[0]
    actual_number_of_trades2 = df_[df_[trade_strategy2] == 1].shape[0]


    for sl in sls:
        for tp in tps:
            if actual_number_of_trades == 0:
                ending_val,win_rate,wins,losses = 0,0,0,0
                ending_val4,win_rate4,wins4,losses4 = 0,0,0,0
            else:
                ending_val,win_rate,wins,losses = strategy_tester_buy(df,col = trade_strategy,sl = sl,tp = tp,num_trades = num_trades,risk_size = risk_size,entry = 'c',pr = False,spread = spread)
                ending_val4,win_rate4,wins4,losses4 = strategy_tester_sell(df,col = trade_strategy,sl = sl,tp = tp,num_trades = num_trades,risk_size = risk_size,entry = 'c',pr = False,spread = spread)   
                
            if actual_number_of_trades2 == 0:
                ending_val2,win_rate2,wins2,losses2 = 0,0,0,0
                ending_val3,win_rate3,wins3,losses3 = 0,0,0,0
            else:
                ending_val2,win_rate2,wins2,losses2 = strategy_tester_sell(df_,col = trade_strategy2,sl = sl,tp = tp,num_trades = num_trades,risk_size = risk_size,entry = 'c',pr = False,spread = spread)
                ending_val3,win_rate3,wins3,losses3 = strategy_tester_buy(df_,col = trade_strategy2,sl = sl,tp = tp,num_trades = num_trades,risk_size = risk_size,entry = 'c',pr = False,spread = spread) 
                
                
            max_ending_val = max(ending_val,
                   ending_val2,
                   ending_val3,
                   ending_val4)
            lst = [instrument,
                   ending_val,
                   ending_val2,
                   ending_val3,
                   ending_val4,
                   max_ending_val,
                    sl,
                    tp,
                    round(tp / sl,4),
                   round(win_rate,4),
                   round(win_rate2,4),
                   round(win_rate3,4),
                   round(win_rate4,4),
                    lookup_range,
                   actual_number_of_trades,
                   actual_number_of_trades2
                  ]
            temp_lst.append(lst)

    final_df = pd.DataFrame(temp_lst,columns = cols)
    final_df = final_df.sort_values(by = 'max_ending_val',ascending = False)
    
    e = time.time()
    best_buy_support = final_df['buy_support_val'].max()
    best_sell_resistance = final_df['sell_resistance_val'].max()
    best_buy_resistance = final_df['buy_resistance_val'].max()
    best_sell_support = final_df['sell_support_val'].max()
    max_val_all = final_df['max_ending_val'].max()
    print('MAX VAL:',round(max_val_all),'||best buy support:',round(best_buy_support),'||best sell resistance:',
          round(best_sell_resistance),'||best buy resistance:',round(best_buy_resistance),'||best sell support:',round(best_sell_support))


    print('TOTAL SINGLE PAIR SIM TIME:',round((e-s)/60,1),'MINUTES')
    return final_df

def run_all_pairs_sim_loop(
                    lookup_range,
    pip_delta,
                    spread,
                    strategy_name = 'large_candle_sim_results'
                    ):
    dir_name = os.getcwd() + '/' + strategy_name 
    print(dir_name)
    isdir = os.path.isdir(dir_name) 
    if isdir:
        pass
    else:
        print('MAKING DIRECTORY:',dir_name)
        os.mkdir(dir_name) 
        
        
    dir_name = os.getcwd() + '/' + strategy_name + '/' + str(lookup_range)
    print(dir_name)
    isdir = os.path.isdir(dir_name) 
    if isdir:
        pass
    else:
        print('MAKING DIRECTORY:',dir_name)
        os.mkdir(dir_name)         
        
        

    if len(os.listdir(dir_name)) > 0:
        print('file already exists in this directory, passing...')
    else:
        print('file does not exist, running sim')


        all_pairs=['EUR_USD','GBP_JPY','USD_JPY','EUR_JPY','AUD_JPY','CAD_JPY','CHF_JPY',
                   'GBP_USD','USD_CAD','USD_CHF','AUD_USD',
                   'NZD_USD','AUD_CAD','AUD_CHF','AUD_NZD','CAD_CHF',
                   'EUR_AUD','EUR_CAD','EUR_CHF','NZD_CAD','NZD_CHF'] 


        final_df = pd.DataFrame([0])    
        for pair in all_pairs:
            print(pair)  

            if 'JPY' in pair:
                sls = [.05,.075,.1,.125,.15,.2,.25]
                tps = [.1,.15,.2,.25,.3,.4,.5,.7,.9,1.1,1.3,1.5]
                spread = .01
                pip_delta_ = pip_delta * 100
            else:
                sls = [.0005,.00075,.001,.00125,.0015,.002,.0025]
                tps = [.001,.0015,.002,.0025,.003,.004,.005,.007,.009,.011,.013,.015]
                spread = .0001      
                pip_delta_ = pip_delta

            df = run_single_pair_sim(pair,
                            lookup_range,
                            pip_delta_,
                            sls,
                            tps,
                            spread)    
            if final_df.shape[0] == 1:
                final_df = df
            else:
                final_df = pd.concat([final_df,df],axis = 0)

        final_df = final_df.sort_values(by = 'max_ending_val',ascending = False)    

        gb = final_df.groupby('instrument').agg({'max_ending_val':max,'buy_support_val':max,'sell_resistance_val':max,'buy_resistance_val':max,'sell_support_val':max})
        gb = gb.reset_index()
        gb = gb.sort_values(by = 'max_ending_val',ascending = False)    
        for i in range(gb.shape[0]):
            print('pair',gb['instrument'].iloc[i],'||MAX VAL',gb['max_ending_val'].iloc[i],'||best buy support:',gb['buy_support_val'].iloc[i],'||best sell resistance:',
                  gb['sell_resistance_val'].iloc[i],'||best buy resistance:',gb['buy_resistance_val'].iloc[i],'||best sell support:',gb['sell_support_val'].iloc[i])

        print('MEDIANS:',np.median(gb['max_ending_val']),np.median(gb['buy_support_val']),np.median(gb['sell_resistance_val']),np.median(gb['buy_resistance_val']),np.median(gb['sell_support_val']))

        path = dir_name + '/' + str(round(np.median(gb['max_ending_val']))) + '_' + str(round(np.median(gb['buy_support_val']))) + '_' + str(round(np.median(gb['sell_resistance_val']))) + '_' + str(round(np.median(gb['buy_resistance_val']))) + '_' + str(round(np.median(gb['sell_support_val'])))
        path = path + '.csv'
        print('FINAL PATH:',path)
        print('SAVING DATA')
        final_df.to_csv(path,index = False)
        print()
        print()
        print()
        print()
        print()
        
        
lookup_range = 75
spread = .0001
pip_delta = .0015
run_all_pairs_sim_loop(
                    lookup_range,
                    pip_delta = pip_delta,
                    spread = spread,
                    strategy_name = 'no_wick_7_on_next_' + str(pip_delta) + 'low_spread_0_sim_results'
                    )



/Users/aidanmcconnell/Documents/MARKET_ANALYTICS/forex/Macro/no_wick_7_on_next_0.0015low_spread_0_sim_results
MAKING DIRECTORY: /Users/aidanmcconnell/Documents/MARKET_ANALYTICS/forex/Macro/no_wick_7_on_next_0.0015low_spread_0_sim_results
/Users/aidanmcconnell/Documents/MARKET_ANALYTICS/forex/Macro/no_wick_7_on_next_0.0015low_spread_0_sim_results/75
MAKING DIRECTORY: /Users/aidanmcconnell/Documents/MARKET_ANALYTICS/forex/Macro/no_wick_7_on_next_0.0015low_spread_0_sim_results/75
file does not exist, running sim
EUR_USD
PATH: /Users/aidanmcconnell/Documents/MARKET_ANALYTICS/forex/Macro/EUR_USD_M5/2016-01-01_2022-07-31.csv
(490023, 26)
(490023, 26)
RUNNING SINGLE PARAMETER LOOP FOR PARAMS: INSTRUMENT: EUR_USD lookup_range 75
GETTING SUPPORT INDICATOR
LOOKUP RANGE AND PIP DELTA: 75 0.0015
COUNT: 0
TOTAL FUNCTION TIME: 0.2965198318163554  MINUTES
SHAPE (20, 27)
GETTING RESISTANCE INDICATOR
LOOKUP RANGE AND PIP DELTA: 75 0.0015
COUNT: 0
TOTAL FUNCTION TIME: 0.02384708325068156  MINUTES
SHAPE 

In [62]:
def test_sim2(df_,acc_val = 10000):
    risk_size = .01
    df = df_.copy()
    lst = []
    for i in range(df.shape[0]):
        rr = df['RR'].iloc[i]
        win_rate = df['test_set_win_rate'].iloc[i]
        num_trades = df['test_set_actual_number_of_trades'].iloc[i]
        for j in range(num_trades):
            lst.append([rr,win_rate,random.random()])
    new_df = pd.DataFrame(lst,columns = ['rr','win_rate','rand'])

    new_df = new_df.sort_values(by = 'rand')
    start = acc_val
    for i in range(new_df.shape[0]):
        win_rate = new_df['win_rate'].iloc[i]
        rr = new_df['rr'].iloc[i]
        rand = random.random()
        if rand > win_rate:
            acc_val = acc_val - (acc_val * risk_size)
        else:
            acc_val = acc_val + (acc_val * (risk_size * rr))
            
    return round(acc_val)  
new_vals = []
for i in range(300):
    if random.random() < .5:
        acc_val = test_sim2(sells,acc_val = 10000)
        acc_val = test_sim2(buys,acc_val = acc_val)
    else:
        acc_val = test_sim2(buys,acc_val = 10000)
        acc_val = test_sim2(sells,acc_val = acc_val)        
    new_vals.append(acc_val)
np.median(new_vals)

115770.0

In [64]:
new_vals = []
for i in range(2000):
    if random.random() < .5:
        acc_val = test_sim2(sells,acc_val = 10000)
        acc_val = test_sim2(buys,acc_val = acc_val)
    else:
        acc_val = test_sim2(buys,acc_val = 10000)
        acc_val = test_sim2(sells,acc_val = acc_val)        
    new_vals.append(acc_val)
np.median(new_vals)

94298.5

In [67]:
df['sl_tp'] = '_'
for i in range(df.shape[0]):
    if 'JPY' in df.iloc[i,0]:
        sl = df['sl'].iloc[i]
        tp = df['tp'].iloc[i]
        sl = sl * 1000
        tp = tp * 1000
    else:
        sl = df['sl'].iloc[i] * 100000
        tp = df['tp'].iloc[i] * 100000
    df.iloc[i,-1] = str(int(sl)) + '_' + str(int(tp))
df.head()

,instrument,buy_support_val,sell_resistance_val,buy_resistance_val,sell_support_val,max_ending_val,sl,tp,RR,win_rate1,win_rate2,win_rate3,win_rate4,lookup_range,support_number_of_trades,resistance_number_of_trades,sl_tp
0,EUR_USD,10539145.0,180.0,180.0,180.0,10539145.0,0.00075,0.015,20.0000,0.1500,0.0000,0.0000,0.0000,75,20,24,75_1500
1,NZD_CAD,6373.0,6283442.5,180.0,180.0,6283442.5,0.00100,0.013,13.0000,0.0741,0.2222,0.0000,0.0000,75,27,18,100_1300
2,NZD_CAD,8469.0,2408875.0,1346.0,2485.0,2408875.0,0.00150,0.013,8.6667,0.1111,0.2778,0.0556,0.0741,75,27,18,150_1300
3,NZD_CAD,4838.5,1774757.0,180.0,180.0,1774757.0,0.00100,0.011,11.0000,0.0741,0.2222,0.0000,0.0000,75,27,18,100_1100
4,NZD_CAD,8338.0,1749417.0,180.0,180.0,1749417.0,0.00100,0.015,15.0000,0.0741,0.1667,0.0000,0.0000,75,27,18,100_1500


In [68]:
gb = df.groupby('sl_tp').agg({'buy_support_val':np.median})
gb = gb.reset_index()
gb = gb.sort_values(by = 'buy_support_val',ascending = False)
gb.head(10)

,sl_tp,buy_support_val
65,50_100,26325.5
78,75_100,23344.5
0,100_100,22517.5
13,125_100,21626.0
26,150_100,18211.0
39,200_100,16370.0
52,250_100,15167.5
40,200_1100,11793.0
41,200_1300,11733.0
61,250_500,10913.5


In [69]:
gb = df.groupby('sl_tp').agg({'buy_support_val':np.average})
gb = gb.reset_index()
gb = gb.sort_values(by = 'buy_support_val',ascending = False)
gb.head(10)

,sl_tp,buy_support_val
82,75_1500,620426.285714
80,75_1300,174232.904762
4,100_1500,89422.238095
69,50_1500,83410.761905
79,75_1100,64524.809524
89,75_899,58480.533333
14,125_1100,53563.523810
76,50_899,52316.800000
27,150_1100,46737.380952
67,50_1300,44695.880952


In [71]:
gb['sl_tp'].iloc[0]

'75_1500'

In [72]:
df.head()

,instrument,buy_support_val,sell_resistance_val,buy_resistance_val,sell_support_val,max_ending_val,sl,tp,RR,win_rate1,win_rate2,win_rate3,win_rate4,lookup_range,support_number_of_trades,resistance_number_of_trades,sl_tp
0,EUR_USD,10539145.0,180.0,180.0,180.0,10539145.0,0.00075,0.015,20.0000,0.1500,0.0000,0.0000,0.0000,75,20,24,75_1500
1,NZD_CAD,6373.0,6283442.5,180.0,180.0,6283442.5,0.00100,0.013,13.0000,0.0741,0.2222,0.0000,0.0000,75,27,18,100_1300
2,NZD_CAD,8469.0,2408875.0,1346.0,2485.0,2408875.0,0.00150,0.013,8.6667,0.1111,0.2778,0.0556,0.0741,75,27,18,150_1300
3,NZD_CAD,4838.5,1774757.0,180.0,180.0,1774757.0,0.00100,0.011,11.0000,0.0741,0.2222,0.0000,0.0000,75,27,18,100_1100
4,NZD_CAD,8338.0,1749417.0,180.0,180.0,1749417.0,0.00100,0.015,15.0000,0.0741,0.1667,0.0000,0.0000,75,27,18,100_1500


In [73]:
def test_set2(df2,pairs,lookup_range,pip_delta,np_median = True):
          
    for pair in pairs:
        trade_strategy = 'support_indicator'
        trade_strategy2 = 'resistance_indicator'

        #trade_strategy = 'resistance_indicatorc'
        num_trades = 400
        risk_size = .01

        starting_val = 10000
        instrument = pair
        timeframe = 'M5'
        temp_lst = []
        df = load_df(pair = pair,granularity ='M5',start = datetime(2010,1,1,0,0,0),end = datetime(2016,1,1,0,0,0))
        print(df.shape)
        df_ = df.copy()
        
        
        if np_median:
            gb = df2.groupby('sl_tp').agg({'buy_support_val':np.median})
            gb = gb.reset_index()
            gb = gb.sort_values(by = 'buy_support_val',ascending = False)

            gb2 = df2.groupby('sl_tp').agg({'sell_resistance_val':np.median})
            gb2 = gb2.reset_index()
            gb2 = gb2.sort_values(by = 'sell_resistance_val',ascending = False)  

            buy_sl = df2[(df2['instrument'] == pair) & (df2['sl_tp'] == gb['sl_tp'].iloc[0])]['sl'].iloc[0]
            buy_tp = df2[(df2['instrument'] == pair) & (df2['sl_tp'] == gb['sl_tp'].iloc[0])]['tp'].iloc[0]

            sell_sl = df2[(df2['instrument'] == pair) & (df2['sl_tp'] == gb2['sl_tp'].iloc[0])]['sl'].iloc[0]
            sell_tp = df2[(df2['instrument'] == pair) & (df2['sl_tp'] == gb2['sl_tp'].iloc[0])]['tp'].iloc[0]

        else:
            gb = df2.groupby('sl_tp').agg({'buy_support_val':np.average})
            gb = gb.reset_index()
            gb = gb.sort_values(by = 'buy_support_val',ascending = False)

            gb2 = df2.groupby('sl_tp').agg({'sell_resistance_val':np.average})
            gb2 = gb2.reset_index()
            gb2 = gb2.sort_values(by = 'sell_resistance_val',ascending = False)  
            
            buy_sl = df2[(df2['instrument'] == pair) & (df2['sl_tp'] == gb['sl_tp'].iloc[0])]['sl'].iloc[0]
            buy_tp = df2[(df2['instrument'] == pair) & (df2['sl_tp'] == gb['sl_tp'].iloc[0])]['tp'].iloc[0]

            sell_sl = df2[(df2['instrument'] == pair) & (df2['sl_tp'] == gb2['sl_tp'].iloc[0])]['sl'].iloc[0]
            sell_tp = df2[(df2['instrument'] == pair) & (df2['sl_tp'] == gb2['sl_tp'].iloc[0])]['tp'].iloc[0]

        print('RUNNING SINGLE PARAMETER LOOP FOR PARAMS:',
              'INSTRUMENT:',pair,
              'lookup_range',lookup_range)

        df = get_support(df,
                        lookup_range = lookup_range,
                         pip_delta = pip_delta
                        )

        df_ = get_resistance(df_,
                        lookup_range = lookup_range,
                         pip_delta = pip_delta
                       )

        actual_number_of_trades = df[df[trade_strategy] == 1].shape[0]
        actual_number_of_trades2 = df_[df_[trade_strategy2] == 1].shape[0]


        if 'JPY' in pair:
            spread = .01
        else:
            spread = .0001
            
        


        ending_val,win_rate,wins,losses = strategy_tester_buy(df,col = trade_strategy,sl = buy_sl,tp = buy_tp,num_trades = num_trades,risk_size = risk_size,entry = 'c',pr = False,spread = spread)
        ending_val2,win_rate2,wins2,losses2 = strategy_tester_sell(df_,col = trade_strategy2,sl = sell_sl,tp = sell_tp,num_trades = num_trades,risk_size = risk_size,entry = 'c',pr = False,spread = spread)

        
        #print("TRAIN SET SIM VALS:",buys.loc[buys[buys['pair'] == pair].index,'sim_val'].iloc[0],sells.loc[sells[sells['pair'] == pair].index,'sim_val'].iloc[0])
        print("TEST SET SIM VALS:",ending_val,ending_val2)
        print()
        print()
        print()


test_set2(df,pairs,lookup_range,pip_delta,np_median = True)



PATH: /Users/aidanmcconnell/Documents/MARKET_ANALYTICS/forex/Macro/EUR_USD_M5/2010-01-01_2016-01-01.csv
(454641, 14)
(454641, 14)
RUNNING SINGLE PARAMETER LOOP FOR PARAMS: INSTRUMENT: EUR_USD lookup_range 75
GETTING SUPPORT INDICATOR
LOOKUP RANGE AND PIP DELTA: 75 0.0015
COUNT: 0
TOTAL FUNCTION TIME: 0.2648635983467102  MINUTES
SHAPE (64, 15)
GETTING RESISTANCE INDICATOR
LOOKUP RANGE AND PIP DELTA: 75 0.0015
COUNT: 0
TOTAL FUNCTION TIME: 0.020885733763376872  MINUTES
SHAPE (48, 15)
TEST SET SIM VALS: 40721.0 145366.5



PATH: /Users/aidanmcconnell/Documents/MARKET_ANALYTICS/forex/Macro/USD_JPY_M5/2010-01-01_2016-01-01.csv
(453159, 14)
(453159, 14)
RUNNING SINGLE PARAMETER LOOP FOR PARAMS: INSTRUMENT: USD_JPY lookup_range 75
GETTING SUPPORT INDICATOR
LOOKUP RANGE AND PIP DELTA: 75 0.0015
COUNT: 0
TOTAL FUNCTION TIME: 0.2708509643872579  MINUTES
SHAPE (327, 15)
GETTING RESISTANCE INDICATOR
LOOKUP RANGE AND PIP DELTA: 75 0.0015
COUNT: 0
TOTAL FUNCTION TIME: 0.021184802055358887  MINUTES
S

In [74]:
test_set2(df,pairs,lookup_range,pip_delta,np_median = False)



PATH: /Users/aidanmcconnell/Documents/MARKET_ANALYTICS/forex/Macro/EUR_USD_M5/2010-01-01_2016-01-01.csv
(454641, 14)
(454641, 14)
RUNNING SINGLE PARAMETER LOOP FOR PARAMS: INSTRUMENT: EUR_USD lookup_range 75
GETTING SUPPORT INDICATOR
LOOKUP RANGE AND PIP DELTA: 75 0.0015
COUNT: 0
TOTAL FUNCTION TIME: 0.2642327189445496  MINUTES
SHAPE (64, 15)
GETTING RESISTANCE INDICATOR
LOOKUP RANGE AND PIP DELTA: 75 0.0015
COUNT: 0
TOTAL FUNCTION TIME: 0.020959063371022543  MINUTES
SHAPE (48, 15)
TEST SET SIM VALS: 4675.0 4166.0



PATH: /Users/aidanmcconnell/Documents/MARKET_ANALYTICS/forex/Macro/USD_JPY_M5/2010-01-01_2016-01-01.csv
(453159, 14)
(453159, 14)
RUNNING SINGLE PARAMETER LOOP FOR PARAMS: INSTRUMENT: USD_JPY lookup_range 75
GETTING SUPPORT INDICATOR
LOOKUP RANGE AND PIP DELTA: 75 0.0015
COUNT: 0
TOTAL FUNCTION TIME: 0.26043858130772907  MINUTES
SHAPE (327, 15)
GETTING RESISTANCE INDICATOR
LOOKUP RANGE AND PIP DELTA: 75 0.0015
COUNT: 0
TOTAL FUNCTION TIME: 0.021050465106964112  MINUTES
SHA

In [54]:
def test_sim2(df_,acc_val = 10000):
    risk_size = .01
    df = df_.copy()
    lst = []
    for i in range(df.shape[0]):
        rr = df['RR'].iloc[i]
        win_rate = df['test_set_win_rate'].iloc[i]
        num_trades = df['test_set_actual_number_of_trades'].iloc[i]
        for j in range(num_trades):
            lst.append([rr,win_rate,random.random()])
    new_df = pd.DataFrame(lst,columns = ['rr','win_rate','rand'])

    new_df = new_df.sort_values(by = 'rand')
    start = acc_val
    for i in range(new_df.shape[0]):
        win_rate = new_df['win_rate'].iloc[i]
        rr = new_df['rr'].iloc[i]
        rand = random.random()
        if rand > win_rate:
            acc_val = acc_val - (acc_val * risk_size)
        else:
            acc_val = acc_val + (acc_val * (risk_size * rr))
            
    return round(acc_val)  


def test_set(pairs,lookup_range,pip_delta,buys,sells):
    for pair in pairs:
        trade_strategy = 'support_indicator'
        trade_strategy2 = 'resistance_indicator'

        #trade_strategy = 'resistance_indicatorc'
        num_trades = 400
        risk_size = .01

        starting_val = 10000
        instrument = pair
        timeframe = 'M5'
        temp_lst = []




        df = load_df(pair = pair,granularity ='M5',start = datetime(2010,1,1,0,0,0),end = datetime(2016,1,1,0,0,0))
        print(df.shape)
        df_ = df.copy()

        print('RUNNING SINGLE PARAMETER LOOP FOR PARAMS:',
              'INSTRUMENT:',pair,
              'lookup_range',lookup_range)

        df = get_support(df,
                        lookup_range = lookup_range,
                         pip_delta = pip_delta
                        )

        df_ = get_resistance(df_,
                        lookup_range = lookup_range,
                         pip_delta = pip_delta
                       )

        actual_number_of_trades = df[df[trade_strategy] == 1].shape[0]
        actual_number_of_trades2 = df_[df_[trade_strategy2] == 1].shape[0]


        if 'JPY' in pair:
            spread = .01
        else:
            spread = .0001
        sl = buys[(buys['pair'] == pair)]['sl'].iloc[0]
        tp = buys[(buys['pair'] == pair)]['tp'].iloc[0]

        ending_val,win_rate,wins,losses = strategy_tester_buy(df,col = trade_strategy,sl = sl,tp = tp,num_trades = num_trades,risk_size = risk_size,entry = 'c',pr = False,spread = spread)
        buys.loc[buys[buys['pair'] == pair].index,'test_set_ending_val'] = ending_val
        buys.loc[buys[buys['pair'] == pair].index,'test_set_win_rate'] = win_rate
        buys.loc[buys[buys['pair'] == pair].index,'test_set_actual_number_of_trades'] = actual_number_of_trades
        
        ev = ending_val

        sl = sells[(sells['pair'] == pair)]['sl'].iloc[0]
        tp = sells[(sells['pair'] == pair)]['tp'].iloc[0]
        ending_val2,win_rate2,wins2,losses2 = strategy_tester_sell(df_,col = trade_strategy2,sl = sl,tp = tp,num_trades = num_trades,risk_size = risk_size,entry = 'c',pr = False,spread = spread)
        sells.loc[sells[sells['pair'] == pair].index,'test_set_ending_val'] = ending_val2
        sells.loc[sells[sells['pair'] == pair].index,'test_set_win_rate'] = win_rate2
        sells.loc[sells[sells['pair'] == pair].index,'test_set_actual_number_of_trades'] = actual_number_of_trades2
        
        print("TRAIN SET SIM VALS:",buys.loc[buys[buys['pair'] == pair].index,'sim_val'].iloc[0],sells.loc[sells[sells['pair'] == pair].index,'sim_val'].iloc[0])
        print("TEST SET SIM VALS:",ending_val,ending_val2)
        print()
        print()
        print()
        
    return buys,sells
        


dir_name = '/Users/aidanmcconnell/Documents/MARKET_ANALYTICS/forex/Macro/no_wick_7_on_next_0.0015low_spread_0_sim_results/75/284319_66220_63794_15841_23212.csv'
df = pd.read_csv(dir_name)
print(df.shape)        
        
pairs = list(df['instrument'].value_counts().index)
buys = []
sells = []
for pair in pairs:
    #temp = df[(df['instrument'] == pair) & (df['RR'] >= 8)]
    temp = df[(df['instrument'] == pair)]
    temp = temp.sort_values(by = 'buy_support_val',ascending = False)

    print('BUY',pair,temp['buy_support_val'].iloc[0],temp['RR'].iloc[0],temp['sl'].iloc[0],temp['tp'].iloc[0],temp['support_number_of_trades'].iloc[0])
    buys.append([pair,temp['buy_support_val'].iloc[0],temp['RR'].iloc[0],temp['sl'].iloc[0],temp['tp'].iloc[0],
                 temp['support_number_of_trades'].iloc[0],round(temp['support_number_of_trades'].iloc[0] * temp['win_rate1'].iloc[0]),0,0,0])

    temp = temp.sort_values(by = 'sell_resistance_val',ascending = False)

    print('SELL',pair,temp['sell_resistance_val'].iloc[0],temp['RR'].iloc[0],temp['sl'].iloc[0],temp['tp'].iloc[0],temp['resistance_number_of_trades'].iloc[0])
    sells.append([pair,temp['sell_resistance_val'].iloc[0],temp['RR'].iloc[0],temp['sl'].iloc[0],temp['tp'].iloc[0],
                  temp['resistance_number_of_trades'].iloc[0],round(temp['resistance_number_of_trades'].iloc[0] * temp['win_rate2'].iloc[0]),0,0,0])
    print()    
buys = pd.DataFrame(buys,columns = ['pair','sim_val','RR','sl','tp','number_of_trades','win_count','test_set_ending_val','test_set_win_rate','test_set_actual_number_of_trades'])
sells = pd.DataFrame(sells,columns = ['pair','sim_val','RR','sl','tp','number_of_trades','win_count','test_set_ending_val','test_set_win_rate','test_set_actual_number_of_trades'])

#pip_delta = .002
buys,sells = test_set(pairs,lookup_range,pip_delta,buys,sells)


new_vals = []
for i in range(300):
    if random.random() < .5:
        acc_val = test_sim2(sells,acc_val = 10000)
        acc_val = test_sim2(buys,acc_val = acc_val)
    else:
        acc_val = test_sim2(buys,acc_val = 10000)
        acc_val = test_sim2(sells,acc_val = acc_val)        
    new_vals.append(acc_val)
np.median(new_vals)



(1764, 16)
BUY EUR_USD 10539145.0 20.0 0.00075 0.015 20
SELL EUR_USD 91578.0 2.0 0.0005 0.001 24

BUY USD_JPY 60814.0 0.6667 0.15 0.1 28
SELL USD_JPY 107871.0 4.0 0.125 0.5 26

BUY AUD_JPY 23232.0 2.0 0.05 0.1 54
SELL AUD_JPY 45106.0 1.3333 0.075 0.1 45

BUY EUR_JPY 27240.0 1.3333 0.075 0.1 87
SELL EUR_JPY 50561.0 5.5 0.2 1.1 62

BUY CAD_JPY 55577.5 7.3333 0.15 1.1 37
SELL CAD_JPY 45116.0 1.0 0.2 0.2 26

BUY USD_CHF 66220.0 2.0 0.0005 0.001 35
SELL USD_CHF 52969.5 30.0 0.0005 0.015 31

BUY EUR_CAD 72887.0 14.0 0.0005 0.007 39
SELL EUR_CAD 78851.0 2.0 0.0005 0.001 50

BUY CHF_JPY 78851.0 2.0 0.05 0.1 41
SELL CHF_JPY 40721.0 2.0 0.05 0.1 31

BUY AUD_CHF 32302.0 18.0 0.0005 0.009 36
SELL AUD_CHF 52632.0 1.3333 0.00075 0.001 22

BUY GBP_JPY 41827.5 2.0 0.05 0.1 78
SELL GBP_JPY 106359.0 2.0 0.05 0.1 67

BUY NZD_USD 131483.5 2.0 0.0005 0.001 29
SELL NZD_USD 56737.0 10.0 0.0005 0.005 31

BUY NZD_CAD 51499.0 7.5 0.002 0.015 27
SELL NZD_CAD 6283442.5 13.0 0.001 0.013 18

BUY USD_CAD 63429.0 13.

NameError: name 'risk_size' is not defined

In [44]:
def add_delta_cols(df):
    #df['delta_vector'] = 0
    #df['delta_max_vector'] = 0

    df['delta_vector'] = 0
    df['delta_max'] = 0
    df['upper_wick'] = 0
    df['lower_wick'] = 0
    
    dv_col = df.columns.get_loc('delta_vector')
    dm_col = df.columns.get_loc('delta_max')
    uw_col = df.columns.get_loc('upper_wick')
    lw_col = df.columns.get_loc('lower_wick')
    
    o_col = df.columns.get_loc('o')
    h_col = df.columns.get_loc('h')
    l_col = df.columns.get_loc('l')
    c_col = df.columns.get_loc('c')
    arr = df.values
    for i in range(arr.shape[0]):
        arr[i,dv_col] = arr[i,c_col] - arr[i,o_col]
        arr[i,dm_col] = arr[i,h_col] - arr[i,l_col]
        if arr[i,c_col] > arr[i,o_col]:
            arr[i,uw_col] = arr[i,h_col] - arr[i,c_col]
            arr[i,lw_col] = arr[i,o_col] - arr[i,l_col]
            
        elif arr[i,c_col] < arr[i,o_col]:
            arr[i,uw_col] = arr[i,h_col] - arr[i,o_col]
            arr[i,lw_col] = arr[i,c_col] - arr[i,l_col]   
            
    df = pd.DataFrame(arr,columns = df.columns)
    df['delta'] = abs(df['delta_vector'])
    
    return df
def load_df(pair = 'EUR_USD'
            ,granularity = 'M5'
            ,start = datetime(2016,1,1,0,0,0)
            ,end = datetime(2022,7,31,0,0,0)):
    def convert_timestamp(df):
        time_col = df.columns.get_loc('time')
        df['minute_col'] = 0
        new_col = df.columns.get_loc('minute_col')
        arr = df.values 
        for i in range(arr.shape[0]):
            arr[i,new_col] = pd.Timestamp(arr[i,time_col]).minute
        return pd.DataFrame(arr,columns = df.columns)
    dir_name = os.getcwd() + '/' + pair + '_' + granularity
    path = dir_name + '/' + str(date(start.year,start.month,start.day)) + '_' + str(date(end.year,end.month,end.day)) + '.csv'
    print('PATH:',path)
    df = pd.read_csv(path)
    df = add_delta_cols(df)
    df = convert_timestamp(df)
    print(df.shape)
    return df
def get_support(df,
                lookup_range = 1500,
                pip_delta = .001
               ):
    print('GETTING SUPPORT INDICATOR')
    print('LOOKUP RANGE AND PIP DELTA:',lookup_range,pip_delta)
    s = time.time()
    c_col = df.columns.get_loc('c')
    o_col = df.columns.get_loc('o')
    l_col = df.columns.get_loc('l')
    h_col = df.columns.get_loc('h')
    d_col = df.columns.get_loc('delta')
    minute_col = df.columns.get_loc('minute_col')
    london_col = df.columns.get_loc('london')
    ny_col = df.columns.get_loc('new_york')
    df['support_indicator'] = 0
    new_col = df.columns.get_loc('support_indicator')
    arr = df.values
    min_count = 0
    c = 0 
    if pip_delta < .01:
        low_spread = 0 #.0001
    else:
        low_spread = 0 #.01
    for i in range(lookup_range,arr.shape[0]):
       # try:
        #DEFINE SUPPORT/RESISTANCE as the MIN or MAX of a lookup range
        df.iloc[i - 11 : i + 1,:]
        if arr[i,c_col] <= min(arr[i - lookup_range : i,c_col]) and arr[i,c_col] <= min(arr[i - 11 : i + 1,l_col]) + low_spread  and (arr[i - 11,o_col] - arr[i,c_col]) >= pip_delta and arr[i,minute_col] == 55 and (arr[i,london_col] == 1 or arr[i,ny_col] == 1):
            arr[i,new_col] = 1

    print('COUNT:',c)

    e = time.time()
    print('TOTAL FUNCTION TIME:',(e-s)/60,' MINUTES')
    df = pd.DataFrame(arr,columns = df.columns)
    print('SHAPE',df[df['support_indicator'] == 1].shape)
    return df

def get_resistance(df,
                lookup_range = 1500,
                    pip_delta = .001
               ):
    print('GETTING RESISTANCE INDICATOR')
    print('LOOKUP RANGE AND PIP DELTA:',lookup_range,pip_delta)
    s = time.time()
    c_col = df.columns.get_loc('c')
    o_col = df.columns.get_loc('o')
    l_col = df.columns.get_loc('l')
    h_col = df.columns.get_loc('h')
    d_col = df.columns.get_loc('delta')
    minute_col = df.columns.get_loc('minute_col')
    london_col = df.columns.get_loc('london')
    ny_col = df.columns.get_loc('new_york')
    df['resistance_indicator'] = 0
    new_col = df.columns.get_loc('resistance_indicator')
    arr = df.values
    min_count = 0
    c = 0 
    if pip_delta < .01:
        low_spread = 0 #.0001
    else:
        low_spread = 0 #.01
    for i in range(lookup_range,arr.shape[0]):
       # try:
        #DEFINE SUPPORT/RESISTANCE as the MIN or MAX of a lookup range
        if arr[i,c_col] >= max(arr[i - lookup_range : i,c_col]) and arr[i,c_col] >= max(arr[i - 11 : i + 1,h_col]) - low_spread and (arr[i,c_col] - arr[i - 11,o_col] ) >= pip_delta and arr[i,minute_col] == 55  and (arr[i,london_col] == 1 or arr[i,ny_col] == 1):
            arr[i,new_col] = 1

    print('COUNT:',c)

    e = time.time()
    print('TOTAL FUNCTION TIME:',(e-s)/60,' MINUTES')
    df = pd.DataFrame(arr,columns = df.columns)
    print('SHAPE',df[df['resistance_indicator'] == 1].shape)
    return df

def run_single_pair_sim(pair,
                lookup_range,
                    pip_delta,
                sls,
                tps,
                spread):
    
    def strategy_tester_buy(df,col = 'support_indicator',sl = .1,tp = .5,num_trades = 300,risk_size = .01,entry = 'c',pr = True,spread = .01):
        def simulate_results(
            num_trades = 200,
            acc_val = 10000,
            rr = 6,
            risk_size = .02,
            win_rate = .2,
            pr = True):

            start = acc_val
            for i in range(num_trades):
                rand = random.random()
                if rand > win_rate:
                    acc_val = acc_val - (acc_val * risk_size)
                else:
                    acc_val = acc_val + (acc_val * (risk_size * rr))
            if pr:
                print('STARTING VAL:',round(start),' ENDING VAL:',round(acc_val))
            return round(acc_val)

        col_num = df.columns.get_loc(col)
        h_col = df.columns.get_loc('h')
        l_col = df.columns.get_loc('l')
        c_col = df.columns.get_loc('c')
        entry_col = df.columns.get_loc(entry)
        arr = df.values
        temp_df = df[df[col] == 1]

        trade_list = []
        trade_res = []
        wins = []
        losses = []
        for i in range(arr.shape[0]):

            if arr[i,col_num] == 1:
                val = arr[i,entry_col]
                trade_id = 0
                try:
                    for j in range(i + 1,i + 10000):
                        if i not in trade_list:
                            if trade_id == 0:
                                if arr[j,l_col] <= val - sl + spread:
                                    trade_res.append(-10)
                                    trade_list.append(i)
                                    losses.append(i)
                                    trade_id = 1

                                    break
                                if arr[j,h_col] >= val + tp:
                                    trade_res.append(50)
                                    trade_list.append(i)
                                    wins.append(i)
                                    trade_id = 1
                                    break
                except:
                    pass
        win_rate = len(wins) / len(trade_list)
        rr = (tp - spread) / (sl + spread)
       # num_trades = len(trade_list)
      #  print('WIN RATE',win_rate,' RR:',rr, 'NUM TRADES:',num_trades,' RISK SIZE:',risk_size)
        vals = []


        acc_val = 10000



        for i in range(30):

            final_val = simulate_results(num_trades = num_trades,acc_val = acc_val,rr = rr,risk_size = risk_size,win_rate = win_rate,pr = pr)
            vals.append(final_val)

        return np.median(vals),win_rate,wins,losses

    def strategy_tester_sell(df,col = 'support_indicator',sl = .1,tp = .5,num_trades = 300,risk_size = .01,entry = 'c',pr = True,spread = .01):
        def simulate_results(
            num_trades = 200,
            acc_val = 10000,
            rr = 6,
            risk_size = .02,
            win_rate = .2,
            pr = True):

            start = acc_val
            for i in range(num_trades):
                rand = random.random()
                if rand > win_rate:
                    acc_val = acc_val - (acc_val * risk_size)
                else:
                    acc_val = acc_val + (acc_val * (risk_size * rr))
            if pr:
                print('STARTING VAL:',round(start),' ENDING VAL:',round(acc_val))
            return round(acc_val)

        col_num = df.columns.get_loc(col)
        h_col = df.columns.get_loc('h')
        l_col = df.columns.get_loc('l')
        c_col = df.columns.get_loc('c')
        entry_col = df.columns.get_loc(entry)
        arr = df.values
        temp_df = df[df[col] == 1]

        trade_list = []
        trade_res = []
        wins = []
        losses = []
        for i in range(arr.shape[0]):

            if arr[i,col_num] == 1:
                val = arr[i,entry_col]
                try:
                    for j in range(i + 1,i + 10000):
                        if i not in trade_list:
                            if arr[j,h_col] >= val + sl - spread:
                                trade_res.append(-10)
                                trade_list.append(i)
                                losses.append(i)
                                break
                            if arr[j,l_col] <= val - tp:
                                trade_res.append(50)
                                trade_list.append(i)
                                wins.append(i)
                                break
                except:
                    pass
        win_rate = len(wins) / len(trade_list)
        rr = (tp - spread) / (sl + spread)
       # num_trades = len(trade_list)
      #  print('WIN RATE',win_rate,' RR:',rr, 'NUM TRADES:',num_trades,' RISK SIZE:',risk_size)
        vals = []


        acc_val = 10000



        for i in range(30):

            final_val = simulate_results(num_trades = num_trades,acc_val = acc_val,rr = rr,risk_size = risk_size,win_rate = win_rate,pr = pr)
            vals.append(final_val)

        return np.median(vals),win_rate,wins,losses


    s = time.time()
    df = load_df(pair = pair,granularity ='M5',start = datetime(2016,1,1,0,0,0),end = datetime(2022,7,31,0,0,0))
    print(df.shape)
    df_ = df.copy()

    print('RUNNING SINGLE PARAMETER LOOP FOR PARAMS:',
          'INSTRUMENT:',pair,
          'lookup_range',lookup_range)

    df = get_support(df,
                    lookup_range = lookup_range,
                     pip_delta = pip_delta
                    )

    df_ = get_resistance(df_,
                    lookup_range = lookup_range,
                     pip_delta = pip_delta
                   )


    cols = ['instrument',
           'buy_support_val',
           'sell_resistance_val',
           'buy_resistance_val',
           'sell_support_val',
            'max_ending_val',
            'sl',
            'tp',
            'RR',
           'win_rate1',
           'win_rate2',
           'win_rate3',
           'win_rate4',
            'lookup_range',
            'support_number_of_trades',
            'resistance_number_of_trades'
          ]

    trade_strategy = 'support_indicator'
    trade_strategy2 = 'resistance_indicator'

    #trade_strategy = 'resistance_indicatorc'
    num_trades = 400
    risk_size = .01

    starting_val = 10000
    instrument = pair
    timeframe = 'M5'
    temp_lst = []
    actual_number_of_trades = df[df[trade_strategy] == 1].shape[0]
    actual_number_of_trades2 = df_[df_[trade_strategy2] == 1].shape[0]


    for sl in sls:
        for tp in tps:
            if actual_number_of_trades == 0:
                ending_val,win_rate,wins,losses = 0,0,0,0
                ending_val4,win_rate4,wins4,losses4 = 0,0,0,0
            else:
                ending_val,win_rate,wins,losses = strategy_tester_buy(df,col = trade_strategy,sl = sl,tp = tp,num_trades = num_trades,risk_size = risk_size,entry = 'c',pr = False,spread = spread)
                ending_val4,win_rate4,wins4,losses4 = strategy_tester_sell(df,col = trade_strategy,sl = sl,tp = tp,num_trades = num_trades,risk_size = risk_size,entry = 'c',pr = False,spread = spread)   
                
            if actual_number_of_trades2 == 0:
                ending_val2,win_rate2,wins2,losses2 = 0,0,0,0
                ending_val3,win_rate3,wins3,losses3 = 0,0,0,0
            else:
                ending_val2,win_rate2,wins2,losses2 = strategy_tester_sell(df_,col = trade_strategy2,sl = sl,tp = tp,num_trades = num_trades,risk_size = risk_size,entry = 'c',pr = False,spread = spread)
                ending_val3,win_rate3,wins3,losses3 = strategy_tester_buy(df_,col = trade_strategy2,sl = sl,tp = tp,num_trades = num_trades,risk_size = risk_size,entry = 'c',pr = False,spread = spread) 
                
                
            max_ending_val = max(ending_val,
                   ending_val2,
                   ending_val3,
                   ending_val4)
            lst = [instrument,
                   ending_val,
                   ending_val2,
                   ending_val3,
                   ending_val4,
                   max_ending_val,
                    sl,
                    tp,
                    round(tp / sl,4),
                   round(win_rate,4),
                   round(win_rate2,4),
                   round(win_rate3,4),
                   round(win_rate4,4),
                    lookup_range,
                   actual_number_of_trades,
                   actual_number_of_trades2
                  ]
            temp_lst.append(lst)

    final_df = pd.DataFrame(temp_lst,columns = cols)
    final_df = final_df.sort_values(by = 'max_ending_val',ascending = False)
    
    e = time.time()
    best_buy_support = final_df['buy_support_val'].max()
    best_sell_resistance = final_df['sell_resistance_val'].max()
    best_buy_resistance = final_df['buy_resistance_val'].max()
    best_sell_support = final_df['sell_support_val'].max()
    max_val_all = final_df['max_ending_val'].max()
    print('MAX VAL:',round(max_val_all),'||best buy support:',round(best_buy_support),'||best sell resistance:',
          round(best_sell_resistance),'||best buy resistance:',round(best_buy_resistance),'||best sell support:',round(best_sell_support))


    print('TOTAL SINGLE PAIR SIM TIME:',round((e-s)/60,1),'MINUTES')
    return final_df

def run_all_pairs_sim_loop(
                    lookup_range,
    pip_delta,
                    spread,
                    strategy_name = 'large_candle_sim_results'
                    ):
    dir_name = os.getcwd() + '/' + strategy_name 
    print(dir_name)
    isdir = os.path.isdir(dir_name) 
    if isdir:
        pass
    else:
        print('MAKING DIRECTORY:',dir_name)
        os.mkdir(dir_name) 
        
        
    dir_name = os.getcwd() + '/' + strategy_name + '/' + str(lookup_range)
    print(dir_name)
    isdir = os.path.isdir(dir_name) 
    if isdir:
        pass
    else:
        print('MAKING DIRECTORY:',dir_name)
        os.mkdir(dir_name)         
        
        

    if len(os.listdir(dir_name)) > 0:
        print('file already exists in this directory, passing...')
    else:
        print('file does not exist, running sim')


        all_pairs=['EUR_USD','GBP_JPY','USD_JPY','EUR_JPY','AUD_JPY','CAD_JPY','CHF_JPY',
                   'GBP_USD','USD_CAD','USD_CHF','AUD_USD',
                   'NZD_USD','AUD_CAD','AUD_CHF','AUD_NZD','CAD_CHF',
                   'EUR_AUD','EUR_CAD','EUR_CHF','NZD_CAD','NZD_CHF'] 


        final_df = pd.DataFrame([0])    
        for pair in all_pairs:
            print(pair)  

            if 'JPY' in pair:
                sls = [.05,.075,.1,.125,.15,.2,.25]
                tps = [.1,.15,.2,.25,.3,.4,.5,.7,.9,1.1,1.3,1.5]
                spread = .01
                pip_delta_ = pip_delta * 100
            else:
                sls = [.0005,.00075,.001,.00125,.0015,.002,.0025]
                tps = [.001,.0015,.002,.0025,.003,.004,.005,.007,.009,.011,.013,.015]
                spread = .0001      
                pip_delta_ = pip_delta

            df = run_single_pair_sim(pair,
                            lookup_range,
                            pip_delta_,
                            sls,
                            tps,
                            spread)    
            if final_df.shape[0] == 1:
                final_df = df
            else:
                final_df = pd.concat([final_df,df],axis = 0)

        final_df = final_df.sort_values(by = 'max_ending_val',ascending = False)    

        gb = final_df.groupby('instrument').agg({'max_ending_val':max,'buy_support_val':max,'sell_resistance_val':max,'buy_resistance_val':max,'sell_support_val':max})
        gb = gb.reset_index()
        gb = gb.sort_values(by = 'max_ending_val',ascending = False)    
        for i in range(gb.shape[0]):
            print('pair',gb['instrument'].iloc[i],'||MAX VAL',gb['max_ending_val'].iloc[i],'||best buy support:',gb['buy_support_val'].iloc[i],'||best sell resistance:',
                  gb['sell_resistance_val'].iloc[i],'||best buy resistance:',gb['buy_resistance_val'].iloc[i],'||best sell support:',gb['sell_support_val'].iloc[i])

        print('MEDIANS:',np.median(gb['max_ending_val']),np.median(gb['buy_support_val']),np.median(gb['sell_resistance_val']),np.median(gb['buy_resistance_val']),np.median(gb['sell_support_val']))

        path = dir_name + '/' + str(round(np.median(gb['max_ending_val']))) + '_' + str(round(np.median(gb['buy_support_val']))) + '_' + str(round(np.median(gb['sell_resistance_val']))) + '_' + str(round(np.median(gb['buy_resistance_val']))) + '_' + str(round(np.median(gb['sell_support_val'])))
        path = path + '.csv'
        print('FINAL PATH:',path)
        print('SAVING DATA')
        final_df.to_csv(path,index = False)
        print()
        print()
        print()
        print()
        print()
        
        
lookup_range = 75
spread = .0001
pip_delta = .0015
run_all_pairs_sim_loop(
                    lookup_range,
                    pip_delta = pip_delta,
                    spread = spread,
                    strategy_name = 'london_no_wick_sim_results_15'
                    )



/Users/aidanmcconnell/Documents/MARKET_ANALYTICS/forex/Macro/london_no_wick_sim_results_15
MAKING DIRECTORY: /Users/aidanmcconnell/Documents/MARKET_ANALYTICS/forex/Macro/london_no_wick_sim_results_15
/Users/aidanmcconnell/Documents/MARKET_ANALYTICS/forex/Macro/london_no_wick_sim_results_15/75
MAKING DIRECTORY: /Users/aidanmcconnell/Documents/MARKET_ANALYTICS/forex/Macro/london_no_wick_sim_results_15/75
file does not exist, running sim
EUR_USD
PATH: /Users/aidanmcconnell/Documents/MARKET_ANALYTICS/forex/Macro/EUR_USD_M5/2016-01-01_2022-07-31.csv
(490023, 26)
(490023, 26)
RUNNING SINGLE PARAMETER LOOP FOR PARAMS: INSTRUMENT: EUR_USD lookup_range 75
GETTING SUPPORT INDICATOR
LOOKUP RANGE AND PIP DELTA: 75 0.0015
COUNT: 0
TOTAL FUNCTION TIME: 0.305934218565623  MINUTES
SHAPE (40, 27)
GETTING RESISTANCE INDICATOR
LOOKUP RANGE AND PIP DELTA: 75 0.0015
COUNT: 0
TOTAL FUNCTION TIME: 0.025954981644948322  MINUTES
SHAPE (25, 27)
MAX VAL: 148344 ||best buy support: 148344 ||best sell resistance: 

In [45]:
lookup_range = 75
spread = .0001
pip_delta = .002
run_all_pairs_sim_loop(
                    lookup_range,
                    pip_delta = pip_delta,
                    spread = spread,
                    strategy_name = 'london_no_wick_sim_results_20'
                    )


/Users/aidanmcconnell/Documents/MARKET_ANALYTICS/forex/Macro/london_no_wick_sim_results_20
MAKING DIRECTORY: /Users/aidanmcconnell/Documents/MARKET_ANALYTICS/forex/Macro/london_no_wick_sim_results_20
/Users/aidanmcconnell/Documents/MARKET_ANALYTICS/forex/Macro/london_no_wick_sim_results_20/75
MAKING DIRECTORY: /Users/aidanmcconnell/Documents/MARKET_ANALYTICS/forex/Macro/london_no_wick_sim_results_20/75
file does not exist, running sim
EUR_USD
PATH: /Users/aidanmcconnell/Documents/MARKET_ANALYTICS/forex/Macro/EUR_USD_M5/2016-01-01_2022-07-31.csv
(490023, 26)
(490023, 26)
RUNNING SINGLE PARAMETER LOOP FOR PARAMS: INSTRUMENT: EUR_USD lookup_range 75
GETTING SUPPORT INDICATOR
LOOKUP RANGE AND PIP DELTA: 75 0.002
COUNT: 0
TOTAL FUNCTION TIME: 0.31297935247421266  MINUTES
SHAPE (23, 27)
GETTING RESISTANCE INDICATOR
LOOKUP RANGE AND PIP DELTA: 75 0.002
COUNT: 0
TOTAL FUNCTION TIME: 0.025615068276723225  MINUTES
SHAPE (14, 27)
MAX VAL: 79048 ||best buy support: 15846 ||best sell resistance: 60

In [26]:
((1000 * 5) / 60) / 24

3.472222222222222

In [9]:
pd.Timestamp(df['utc_timestamp'].iloc[11]).minute

55

In [19]:
def convert_timestamp(df):
    time_col = df.columns.get_loc('time')
    df['minute_col'] = 0
    new_col = df.columns.get_loc('minute_col')
    arr = df.values 
    for i in range(arr.shape[0]):
        arr[i,new_col] = pd.Timestamp(arr[i,time_col]).minute
    return pd.DataFrame(arr,columns = df.columns)
df = convert_timestamp(df)
df.head()

,time,volume,o,h,l,c,utc_timestamp,est_timestamp,smma_21,smma_50,...,bearish_tls,bullish_tls,new_york,london,delta_vector,delta_max,upper_wick,lower_wick,delta,minute_col
0,2016-01-03 22:00:00,6,1.08743,1.08746,1.08724,1.08726,2016-01-03 22:00:00+00:00,2016-01-03 17:00:00-05:00,1.08726,1.08726,...,0,0,0,0,-0.00017,0.00022,0.00003,0.00002,0.00017,0
1,2016-01-03 22:05:00,53,1.08735,1.08738,1.0871,1.0871,2016-01-03 22:05:00+00:00,2016-01-03 17:05:00-05:00,1.0871,1.0871,...,0,0,0,0,-0.00025,0.00028,0.00003,0.0,0.00025,5
2,2016-01-03 22:10:00,21,1.08715,1.08715,1.08674,1.08681,2016-01-03 22:10:00+00:00,2016-01-03 17:10:00-05:00,1.08681,1.08681,...,0,0,0,0,-0.00034,0.00041,0.0,0.00007,0.00034,10
3,2016-01-03 22:15:00,19,1.08671,1.08694,1.08669,1.0868,2016-01-03 22:15:00+00:00,2016-01-03 17:15:00-05:00,1.0868,1.0868,...,0,0,0,0,0.00009,0.00025,0.00014,0.00002,0.00009,15
4,2016-01-03 22:20:00,47,1.08684,1.08717,1.0867,1.0867,2016-01-03 22:20:00+00:00,2016-01-03 17:20:00-05:00,1.0867,1.0867,...,0,0,0,0,-0.00014,0.00047,0.00033,0.0,0.00014,20


In [11]:
df['time'].iloc[0]

'2016-01-03 22:00:00'

In [27]:
df.head()

,time,volume,o,h,l,c,utc_timestamp,est_timestamp,smma_21,smma_50,...,bearish_tls,bullish_tls,new_york,london,delta_vector,delta_max,upper_wick,lower_wick,delta,minute_col
0,2016-01-03 22:00:00,6,1.08743,1.08746,1.08724,1.08726,2016-01-03 22:00:00+00:00,2016-01-03 17:00:00-05:00,1.08726,1.08726,...,0,0,0,0,-0.00017,0.00022,0.00003,0.00002,0.00017,0
1,2016-01-03 22:05:00,53,1.08735,1.08738,1.0871,1.0871,2016-01-03 22:05:00+00:00,2016-01-03 17:05:00-05:00,1.0871,1.0871,...,0,0,0,0,-0.00025,0.00028,0.00003,0.0,0.00025,5
2,2016-01-03 22:10:00,21,1.08715,1.08715,1.08674,1.08681,2016-01-03 22:10:00+00:00,2016-01-03 17:10:00-05:00,1.08681,1.08681,...,0,0,0,0,-0.00034,0.00041,0.0,0.00007,0.00034,10
3,2016-01-03 22:15:00,19,1.08671,1.08694,1.08669,1.0868,2016-01-03 22:15:00+00:00,2016-01-03 17:15:00-05:00,1.0868,1.0868,...,0,0,0,0,0.00009,0.00025,0.00014,0.00002,0.00009,15
4,2016-01-03 22:20:00,47,1.08684,1.08717,1.0867,1.0867,2016-01-03 22:20:00+00:00,2016-01-03 17:20:00-05:00,1.0867,1.0867,...,0,0,0,0,-0.00014,0.00047,0.00033,0.0,0.00014,20


In [35]:
df.iloc[i - 11,:]

time                       2016-01-03 22:00:00
volume                                       6
o                                      1.08743
h                                      1.08746
l                                      1.08724
c                                      1.08726
utc_timestamp        2016-01-03 22:00:00+00:00
est_timestamp        2016-01-03 17:00:00-05:00
smma_21                                1.08726
smma_50                                1.08726
smma_200                               1.08726
ema_42                                 1.08726
ema_150                                1.08726
ema_600                                1.08726
bearish_engulfing                            0
bullish_engulfing                            0
bearish_tls                                  0
bullish_tls                                  0
new_york                                     0
london                                       0
delta_vector                          -0.00017
delta_max    

In [34]:
i = 11
df.iloc[i - 11 : i + 1,:]

,time,volume,o,h,l,c,utc_timestamp,est_timestamp,smma_21,smma_50,...,bearish_tls,bullish_tls,new_york,london,delta_vector,delta_max,upper_wick,lower_wick,delta,minute_col
0,2016-01-03 22:00:00,6,1.08743,1.08746,1.08724,1.08726,2016-01-03 22:00:00+00:00,2016-01-03 17:00:00-05:00,1.08726,1.08726,...,0,0,0,0,-0.00017,0.00022,0.00003,0.00002,0.00017,0
1,2016-01-03 22:05:00,53,1.08735,1.08738,1.0871,1.0871,2016-01-03 22:05:00+00:00,2016-01-03 17:05:00-05:00,1.0871,1.0871,...,0,0,0,0,-0.00025,0.00028,0.00003,0.0,0.00025,5
2,2016-01-03 22:10:00,21,1.08715,1.08715,1.08674,1.08681,2016-01-03 22:10:00+00:00,2016-01-03 17:10:00-05:00,1.08681,1.08681,...,0,0,0,0,-0.00034,0.00041,0.0,0.00007,0.00034,10
3,2016-01-03 22:15:00,19,1.08671,1.08694,1.08669,1.0868,2016-01-03 22:15:00+00:00,2016-01-03 17:15:00-05:00,1.0868,1.0868,...,0,0,0,0,0.00009,0.00025,0.00014,0.00002,0.00009,15
4,2016-01-03 22:20:00,47,1.08684,1.08717,1.0867,1.0867,2016-01-03 22:20:00+00:00,2016-01-03 17:20:00-05:00,1.0867,1.0867,...,0,0,0,0,-0.00014,0.00047,0.00033,0.0,0.00014,20
5,2016-01-03 22:25:00,13,1.08667,1.08686,1.08667,1.08686,2016-01-03 22:25:00+00:00,2016-01-03 17:25:00-05:00,1.08686,1.08686,...,0,0,0,0,0.00019,0.00019,0.0,0.0,0.00019,25
6,2016-01-03 22:30:00,57,1.08689,1.0872,1.08684,1.08694,2016-01-03 22:30:00+00:00,2016-01-03 17:30:00-05:00,1.08694,1.08694,...,0,0,0,0,0.00005,0.00036,0.00026,0.00005,0.00005,30
7,2016-01-03 22:35:00,3,1.08696,1.08696,1.08691,1.08691,2016-01-03 22:35:00+00:00,2016-01-03 17:35:00-05:00,1.08691,1.08691,...,0,0,0,0,-0.00005,0.00005,0.0,0.0,0.00005,35
8,2016-01-03 22:40:00,3,1.08688,1.08688,1.08682,1.08682,2016-01-03 22:40:00+00:00,2016-01-03 17:40:00-05:00,1.08682,1.08682,...,0,0,0,0,-0.00006,0.00006,0.0,0.0,0.00006,40
9,2016-01-03 22:45:00,44,1.08685,1.08692,1.08668,1.08668,2016-01-03 22:45:00+00:00,2016-01-03 17:45:00-05:00,1.08668,1.08668,...,0,0,0,0,-0.00017,0.00024,0.00007,0.0,0.00017,45


In [28]:
df[df.minute_col == 55]

,time,volume,o,h,l,c,utc_timestamp,est_timestamp,smma_21,smma_50,...,bearish_tls,bullish_tls,new_york,london,delta_vector,delta_max,upper_wick,lower_wick,delta,minute_col
11,2016-01-03 22:55:00,3,1.08668,1.08674,1.08668,1.0867,2016-01-03 22:55:00+00:00,2016-01-03 17:55:00-05:00,1.0867,1.0867,...,0,0,0,0,0.00002,0.00006,0.00004,0.0,0.00002,55
23,2016-01-03 23:55:00,40,1.0852,1.08528,1.08518,1.08519,2016-01-03 23:55:00+00:00,2016-01-03 18:55:00-05:00,1.086102,1.08519,...,0,0,0,0,-0.00001,0.0001,0.00008,0.00001,0.00001,55
35,2016-01-04 00:55:00,51,1.08456,1.08463,1.08446,1.08454,2016-01-04 00:55:00+00:00,2016-01-03 19:55:00-05:00,1.085572,1.08454,...,0,0,0,0,-0.00002,0.00017,0.00007,0.00008,0.00002,55
47,2016-01-04 01:55:00,148,1.0857,1.08617,1.08568,1.08597,2016-01-04 01:55:00+00:00,2016-01-03 20:55:00-05:00,1.085017,1.08597,...,0,0,0,0,0.00027,0.00049,0.0002,0.00002,0.00027,55
59,2016-01-04 02:55:00,98,1.08674,1.0875,1.08668,1.08716,2016-01-04 02:55:00+00:00,2016-01-03 21:55:00-05:00,1.085701,1.085572,...,0,0,0,0,0.00042,0.00082,0.00034,0.00006,0.00042,55
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
489974,2022-08-03 05:55:00,469,1.01774,1.01782,1.01744,1.01764,2022-08-03 05:55:00+00:00,2022-08-03 01:55:00-04:00,1.018005,1.017748,...,0,0,0,0,-0.0001,0.00038,0.00008,0.0002,0.0001,55
489986,2022-08-03 06:55:00,576,1.0157,1.01599,1.01542,1.01598,2022-08-03 06:55:00+00:00,2022-08-03 02:55:00-04:00,1.017478,1.017558,...,0,0,0,0,0.00028,0.00057,0.00001,0.00028,0.00028,55
489998,2022-08-03 07:55:00,541,1.01818,1.01848,1.01772,1.0178,2022-08-03 07:55:00+00:00,2022-08-03 03:55:00-04:00,1.017636,1.017609,...,0,0,0,1,-0.00038,0.00076,0.0003,0.00008,0.00038,55
490010,2022-08-03 08:55:00,483,1.0187,1.01887,1.01842,1.01881,2022-08-03 08:55:00+00:00,2022-08-03 04:55:00-04:00,1.018021,1.017798,...,0,0,0,1,0.00011,0.00045,0.00006,0.00028,0.00011,55


In [ ]:
lookup_range = 1000
spread = .0001

run_all_pairs_sim_loop(
                    lookup_range,
                    spread,
                    strategy_name = 'no_wick_sim_results'
                    )